# Load Results from Previous Sessions

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/DESI_SciencePaperConfirmation/kmax_0.1h/ps4.db')

# Plot Set 1 - Match Neutrino/Relic Forecasts

# GOOD

For some cosmology, we wish to show that the fisher forecast produces identical constraints for neutrinos and for an equivalent relic.  

### OPERATIONAL NOTE: 
We want to add the same CMB information to both the Neutrino and Equivalent Relic forecasts. If I start with the same CMB Fisher matrix, where one row/column is dlogP/dM_ncdm, I must appropriately scale it before use as the CMB prior for relics by multiplying that row/column by the factor 3^(3/4). 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync1/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps1_resultsdir = os.path.join(projectdir, 'results', 'ps1')
ps1_resultsdir_neutrino = os.path.join(ps1_resultsdir, 'neutrino')
ps1_resultsdir_relic = os.path.join(ps1_resultsdir, 'relic')
ps1_convergencedir_neutrino = os.path.join(ps1_resultsdir, 'convergence', 'neutrino')
ps1_convergencedir_relic = os.path.join(ps1_resultsdir, 'convergence', 'relic')
cf.makedirectory(ps1_resultsdir_neutrino)
cf.makedirectory(ps1_resultsdir_relic)
cf.makedirectory(ps1_convergencedir_neutrino) 
cf.makedirectory(ps1_convergencedir_relic) 

# Fiducial Cosmologies
ps1_neutrinofid = {
        "A_s" : 2.22e-9, 
        "n_s" : 0.965,
        "omega_b" : 0.02222,
        "omega_cdm" : 0.1120,
        "tau_reio" : 0.06,
        "h" : 0.70,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 3., 
        "T_ncdm" : 1.95/2.726, 
        "m_ncdm" : 0.03, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_vary" : None #Not used for neutrino forecasts
        } 
# For matching the neutrino/relic forecasts, we must modify temp/mass according to equation 44 
# of the Cora/Julian paper. That is: 
# m_chi = m_nu * 3^(1/4) ***where m_nu is the mass of a single neutrino in the degenerate hierarchy
# T_chi = 1.95 * 3^(1/4)
ps1_relicfid = {
        "A_s" : 2.22e-9, 
        "n_s" : 0.965,
        "omega_b" : 0.02222,
        "omega_cdm" : 0.1120,
        "tau_reio" : 0.06,
        "h" : 0.70,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (1.95/2.726) * np.power(3., 1./4.), # Units [T_cmb]  Are Julian's matrices at 1.64 K? 
        "m_ncdm" : 0.03 * np.power(3., 1./4.), # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_vary" : "m_ncdm" # Fix T_ncdm or M_ncdm 
        }

# DESI survey parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

#Demonstrate Convergence
#neutrinoconvergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'neutrino', # 'relic' or 'neutrino' forecasting scheme 
#    ps1_neutrinofid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'M_ncdm',                                                                  
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fcoverage_deg=14000, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps1_convergencedir_neutrino, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2']
#    )
#neutrinoconvergencetest.gen_all_plots() # Display convergence plots
#relicconvergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'relic', # 'relic' or 'neutrino' forecasting scheme 
#    ps1_relicfid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'M_ncdm',                                                                  
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fcoverage_deg=14000, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps1_convergencedir_relic, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2']
#    )
#relicconvergencetest.gen_all_plots() # Display convergence plots

# Run forecasts
neutrinoforecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    ps1_neutrinofid, 
    z_table, 
    "DESI", 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)
neutrinoforecast.gen_pm()
neutrinoforecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                           
        'omega_ncdm',                                 
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'],
    mu_step=mu_integral_step, 
    skipgen=False)


In [ ]:
relicforecast = cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    ps1_relicfid, 
    z_table,
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)
relicforecast.gen_pm()
relicforecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'omega_ncdm',
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'],
    mu_step=mu_integral_step, 
    skipgen=False)

dill.dump_session(os.path.join(ps1_resultsdir, 'ps1.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps1/ps1.db')

# Add CMB data
neutrinoforecast.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'omega_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"),
    )

relicforecast.load_cmb_fisher( # We are specifically adding the CMBS4 fisher information for neutrinos
    fisher_order=[             # for the purpose of making a comparison, though we should add the
        'omega_b',             # information for relics in the real forecast                          
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'omega_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Relics.dat"),
    )

# Save results
neutrinoforecast.export_matrices(ps1_resultsdir_neutrino)
relicforecast.export_matrices(ps1_resultsdir_relic)

In [ ]:
# MAKE PLOTS
# Show plots inline, and load main getdist plot module and samples class
from __future__ import print_function
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys, os
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
from getdist import plots, MCSamples, loadMCSamples
from getdist.gaussian_mixtures import GaussianND
import getdist
# use this *after* importing getdist if you want to use interactive plots
# %matplotlib notebook
import matplotlib.pyplot as plt
import IPython
import numpy as np
import pandas as pd
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))
# matplotlib 2 may not work very well without usetex on, can uncomment
# plt.rcParams['text.usetex']=True

paramnames = [r'$\omega_b$', r'$\omega_{cdm}$', r'$n_s$', r'$A_s$', r'$\tau_{reio}$', 'h', r'$\omega_{ncdm}$', r'$\sigma_{fog}$', r'$\beta_0$', r'$\alpha_{k2}$']
fiducials = [0.02222, 0.1120, 0.965, 2.22e-9, 0.06, 0.7, 0.09/93.14, 250000, 1.0, 1.0]
#paramnames = [r'$\omega_b$', r'$\omega_{cdm}$', r'$n_s$', r'$A_s$', r'$\tau_{reio}$', 'h', r'$\omega_\chi$']
#fiducials = [0.02222, 0.1120, 0.965, 2.22e-9, 0.06, 0.7, 0.09/93.14]

reliccovmat = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/PAPER_RESULTS/ps1/relic/inv_fullfisher.mat", skiprows=1)
neutrinocovmat = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/PAPER_RESULTS/ps1/neutrino/inv_fullfisher.mat", skiprows=1)

relicgauss = GaussianND(fiducials, reliccovmat, names=paramnames)
neutrinogauss = GaussianND(fiducials, neutrinocovmat, names=paramnames)

g = plots.get_subplot_plotter()
g.settings.axes_fontsize=20
g.settings.axes_labelsize=28
g.settings.axis_tick_x_rotation=45
g.settings.legend_fontsize=28
g.triangle_plot(
    [relicgauss, neutrinogauss], 
    #filled=True,
    legend_labels=['Equivalent Relic', 'Neutrino'],
    legend_loc='upper right',
    line_args = [
        {'ls':'dotted', 'lw':2, 'color':'#A51C30'},
        {'ls':'--', 'lw':2, 'color':'#93a1ad'}],
    contour_args=[
        {'ls':'dotted', 'lw':2, 'color':'#A51C30', 'filled' : False},
        {'ls':'--', 'lw':2, 'color':'#93a1ad', 'filled' : False}])
#plt.suptitle('Note: Same CMB fisher used for both cases.', fontsize=16)
plt.savefig("/Users/nicholasdeporzio/Desktop/relicneutrino_match.png")

# Plot Set 2 - Compare Neutrino Forecast with MCMC for DESI

We wish to generate a LCDM forecast for the DESI 2D posteriors overplotted on MCMC contours... We should forecast for:
1. relic steps on  
2. relic steps off 
3. shift in parameter values due to relics steps on/off 
4. overplot (1)-(3) on MCMC contours 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill

print(cf.RSD_DELTA_L_NUMERATOR_FACTOR)
print(cf.RSD_DELTA_LAMBDACDM)


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync2/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps2_resultsdir = os.path.join(projectdir, 'results', 'ps2')
ps2_convergencedir = os.path.join(ps2_resultsdir, 'convergence')
cf.makedirectory(ps2_resultsdir)
cf.makedirectory(ps2_convergencedir) 


# No Relicfast CMBS4_desi_DH 2020-01-30 Chains Bestfit PLUS Forecasted Shift for relicfast
ps2_fid = { 
        "A_s" : 2.234591e-09-4.806938031745625e-12, 
        "n_s" : 9.659302e-01+5.311035111640709e-05,
        "omega_b" : 2.227047e-02+1.1602668787041021e-06,
        "omega_cdm" : 1.127401e-01-4.108796815856762e-05,
        "tau_reio" : 6.019503e-02-0.001064158458315645,
        "h" : 7.007968e-01+0.0010625065009387986,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 3., 
        "T_ncdm" : 1.95/2.726, 
        "m_ncdm" : 3.623808e-02-(0.009823889445796/3), # Units [eV]
        "b0" : 1.003223e-01-0.004980685985104448, 
        "alphak2" : 9.948011e+00+0.10341895410597579,
        "sigma_fog_0" : 2.497568e+05+762.6869526527007, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_vary" : None, #Not used for neutrino forecasts
        }



# DESI survey parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

#Demonstrate Convergence
#ps2_convergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'neutrino', # 'relic' or 'neutrino' forecasting scheme 
#    ps2_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'M_ncdm',                                                                  
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fcoverage_deg=14000, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps2_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2'] 
#    )
#ps2_convergencetest.gen_all_plots() # Display convergence plots

# Run Fisher Forecast for cosmology
ps2_forecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    ps2_fid, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover,
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)
ps2_forecast.gen_pm()
ps2_forecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                  
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ],
    mu_step=mu_integral_step, 
    skipgen=False)
dill.dump_session(os.path.join(ps2_resultsdir, 'ps2.db'))

In [ ]:
#import dill
#dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps2/ps2.db')

# Add CMB information
ps2_forecast.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'M_ncdm'],
        fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"), 
        colnames=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'M_ncdm'])

# Save results
ps2_forecast.export_matrices(ps2_resultsdir)


# Print errors
for i in range(10):
    print(ps2_forecast.fisher_order[i], "uncertainty: ", np.sqrt(ps2_forecast.numpy_full_covariance[i, i]))

# Plot Set 3 - Compare Neutrino Forecast with MCMC for EUCLID

We wish to generate a LCDM forecast for the EUCLID 2D posteriors overplotted on MCMC contours... We should forecast for:
1. relic steps on  
2. relic steps off 
3. shift in parameter values due to relics steps on/off 
4. overplot (1)-(3) on MCMC contours 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync3/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps3_resultsdir = os.path.join(projectdir, 'results', 'ps3')
ps3_convergencedir = os.path.join(ps3_resultsdir, 'convergence')
cf.makedirectory(ps3_resultsdir)
cf.makedirectory(ps3_convergencedir) 



# EUCLID survey parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, 
    2308.975, 1514.831, 1474.707, 893.716, 497.613])
skycover = 0.3636 # Sky coverage of survey in fraction

#Demonstrate Convergence
#ps3_convergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'neutrino', # 'relic' or 'neutrino' forecasting scheme 
#    ps3_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'M_ncdm',                                                                  
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fsky=skycover, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps3_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2'] 
#    )
#ps3_convergencetest.gen_all_plots() # Display convergence plots

# Run Fisher Forecast
ps3_forecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    ps3_fid, 
    z_table, 
    "EUCLID", 
    dNdz, 
    fsky=skycover,
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)
ps3_forecast.gen_pm()
ps3_forecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'beta0',
        'beta1',                                       
        'alpha_k2'
    ],
    mu_step=mu_integral_step, 
    skipgen=False)

dill.dump_session(os.path.join(ps3_resultsdir, 'ps3.db'))

# Add CMB information
ps3_forecast.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'M_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"),
    colnames=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'H_0',                                                                             
        'M_ncdm'])

# Save results
ps3_forecast.export_matrices(ps3_resultsdir)

# Print errors
for i in range(10):
    print(ps3_forecast.fisher_order[i], "uncertainty: ", np.sqrt(ps3_forecast.numpy_full_covariance[i, i]))

In [ ]:
import dill
dill.load_session("/Users/nicholasdeporzio/Desktop/cfworkspace/results/NeutrinoMatch_CMBS4_euclid_beta_3_DH/ps3_CMBS4_norelicfast_beta3bestfit/ps3.db")

In [ ]:
ps3_fid

# Plot Set 4 - Relic Forecast for DESI at 1.5072 [K] 

We wish to plot the uncertainty in relic temperature as a function of relic mass for masses on the range of 10^-1 to 10^2 [eV] at fiducial relic temperature 1.5072 [K] with the DESI survey. 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync4/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 


# Generate output paths  
ps4_resultsdir = os.path.join(projectdir, 'results', 'ps4')
ps4_convergencedir = os.path.join(ps4_resultsdir, 'convergence')
cf.makedirectory(ps4_resultsdir)
cf.makedirectory(ps4_convergencedir) 

# Linda Fiducial Cosmology 
ps4_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "T_ncdm" # Fix T_ncdm or m_ncdm 
        }

# DESI Parameters
z_table = z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Demonstrate Convergence
#ps4_convergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'relic', # 'relic' or 'neutrino' forecasting scheme 
#    ps4_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'T_ncdm', 
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fcoverage_deg=skycover, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps4_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2'] 
#    )
#ps4_convergencetest.gen_all_plots()

# Run Fisher Forecast
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps4_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps4_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps4_fiducialset = [dict(ps4_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps4_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps4_fiducialset)]
for fidx, fcst in enumerate(ps4_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm',                                 
            'sigma_fog',                                   
            'b0',                                         
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")

In [ ]:
dill.dump_session(os.path.join(ps4_resultsdir, 'ps4.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps4/Low_Masses/Fixed_Kmax/ps4.db')

# Save results 
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t b0 \t alpha_k2'
for fidx, fval in enumerate(ps4_forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm'],
        fisherpath=inpath+str(fidx+1)+"_T1.5K.txt"
    )
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    outdir=os.path.join(ps4_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + '_Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)

# Plot Set 5 - Relic Forecast for DESI at 0.9052 [K]

We wish to plot the uncertainty in relic temperature as a function of relic mass for masses on the range of 10^-1 to 10^2 [eV] at fiducial relic temperature 0.9052 [K] with the  DESI  survey. 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync5/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps5_resultsdir = os.path.join(projectdir, 'results', 'ps5')
ps5_convergencedir = os.path.join(ps5_resultsdir, 'convergence')
cf.makedirectory(ps5_resultsdir)
cf.makedirectory(ps5_convergencedir) 

# Set fiducial cosmology 
# Linda Fiducial Cosmology 
ps5_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (0.9052/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "T_ncdm" # Fix T_ncdm or m_ncdm 
        }

# DESI Parameters
z_table = z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Demonstrate Convergence
#ps5_convergencetest = cf.convergence(
#   classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'relic', # 'relic' or 'neutrino' forecasting scheme 
#    ps5_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'T_ncdm', 
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fcoverage_deg=skycover, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps5_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2'] 
#    )
#ps5_convergencetest.gen_all_plots()

# Run Fisher Forecast
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps5_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps5_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps5_fiducialset = [dict(ps5_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps5_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps5_fiducialset)]
for fidx, fcst in enumerate(ps5_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm',                                 
            'sigma_fog',                                   
            'b0',                                         
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")

In [ ]:
dill.dump_session(os.path.join(ps5_resultsdir, 'ps5.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps5/Low_Masses/Fixed_Kmax/ps5.db')

# Save results 
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t b0 \t alpha_k2'
for fidx, fval in enumerate(ps5_forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm'],
        fisherpath=inpath+str(fidx+1)+"_T0.9K.txt"
    )
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    outdir=os.path.join(ps5_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + 'Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)

# Plot Set 6 - Relic Forecast for EUCLID at 1.5072 [K]

We wish to plot the uncertainty in relic temperature as a function of relic mass for masses on the range of 10^-1 to 10^2 [eV] at fiducial relic temperature 1.5072 [K] with the EUCLID survey. 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync6/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 


# Generate output paths  
ps6_resultsdir = os.path.join(projectdir, 'results', 'ps6')
ps6_convergencedir = os.path.join(ps6_resultsdir, 'convergence')
cf.makedirectory(ps6_resultsdir)
cf.makedirectory(ps6_convergencedir) 

# Set fiducial cosmology 
# Linda Fiducial Cosmology 
ps6_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "beta0" : 1.7, 
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "T_ncdm" # Fix T_ncdm or m_ncdm 
        }

# EUCLID values
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, 
    2308.975, 1514.831, 1474.707, 893.716, 497.613])
skycover = 0.3636 # Sky coverage of survey in fraction

# Demonstrate Convergence
#ps6_convergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'relic', # 'relic' or 'neutrino' forecasting scheme 
#    ps6_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    "EUCLID",
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'T_ncdm', 
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fsky=skycover, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps6_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'beta0',
#                'beta1',
#                'alphak2'] 
#    )
#ps6_convergencetest.gen_all_plots()

# Run Fisher Forecast
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps6_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps6_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps6_fiducialset = [dict(ps6_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps6_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "EUCLID",
    dNdz, 
    fsky=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps6_fiducialset)]
for fidx, fcst in enumerate(ps6_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm',                                 
            'sigma_fog',                                   
            'beta0',
            'beta1',
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")

In [ ]:
dill.dump_session(os.path.join(ps6_resultsdir, 'ps6.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps6/Low_Masses/Fixed_Kmax/ps6.db')

# Save results 
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'
for fidx, fval in enumerate(ps6_forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm'],
        fisherpath=inpath+str(fidx+1)+"_T1.5K.txt"
    )
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    outdir=os.path.join(ps6_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + 'Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)
    

# Plot Set 7 - Relic Forecast for EUCLID at 0.9052 [K]

We wish to plot the uncertainty in relic temperature as a function of relic mass for masses on the range of 10^-1 to 10^2 [eV] at fiducial relic temperature 0.9052 [K] with the EUCLID survey. 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync7/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
#mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 
mu_integral_step = 1.0

# Generate output paths  
ps7_resultsdir = os.path.join(projectdir, 'results', 'ps7')
ps7_convergencedir = os.path.join(ps7_resultsdir, 'convergence')
cf.makedirectory(ps7_resultsdir)
cf.makedirectory(ps7_convergencedir) 

# Linda Fiducial Cosmology 
ps7_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (0.9052/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "beta0" : 1.7, 
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "T_ncdm" # Fix T_ncdm or m_ncdm 
        }

# EUCLID values
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, 
    2308.975, 1514.831, 1474.707, 893.716, 497.613])
skycover = 0.3636 # Sky coverage of survey in fraction

#Demonstrate Convergence
#ps7_convergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'relic', # 'relic' or 'neutrino' forecasting scheme 
#    ps7_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'T_ncdm', 
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fsky=skycover, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps7_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2'] 
#    )
#ps7_convergencetest.gen_all_plots()

# Run Fisher Forecast
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps7_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps7_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps7_fiducialset = [dict(ps7_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps7_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "EUCLID",
    dNdz, 
    fsky=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps7_fiducialset)]
for fidx, fcst in enumerate(ps7_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[                                                          
            'omega_b',                                                          
            'omega_cdm',                                                        
            'n_s',                                                              
            'A_s',                                                              
            'tau_reio',                                                         
            'h',                                                                
            'T_ncdm',                                                           
            'sigma_fog',                                                        
            'beta0',
            'beta1',
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False
    )
    print("Relic Forecast ", fidx, " complete...")

In [ ]:
dill.dump_session(os.path.join(ps7_resultsdir, 'ps7.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps7/Low_Masses/Fixed_Kmax/ps7.db')

# Save results 
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t omega_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'
for fidx, fval in enumerate(ps7_forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm'],
        fisherpath=inpath+str(fidx+1)+"_T0.9K.txt"
    )
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    outdir=os.path.join(ps7_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + 'Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)

# Plot Set 8 - Relic Forecast for BOSS at 1.5072[K]

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync8/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps8_resultsdir = os.path.join(projectdir, 'results', 'ps8')
ps8_convergencedir = os.path.join(ps8_resultsdir, 'convergence')
cf.makedirectory(ps8_resultsdir)
cf.makedirectory(ps8_convergencedir) 

# Set fiducial cosmology 
# Linda Fiducial Cosmology 
ps8_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "beta0" : 1.7, 
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "T_ncdm" # Fix T_ncdm or m_ncdm 
        }

# BOSS values
z_table = np.array([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75])
dNdz = np.array([8., 50., 125., 222., 332., 447., 208., 30.])
skycover = 10000. # Sky coverage of BOSS in degrees^2

#Demonstrate Convergence
#ps8_convergencetest = cf.convergence(
#    classpath, # Path to CLASS installation
#    datastore, # Path to directory holding CLASS output data
#    'relic', # 'relic' or 'neutrino' forecasting scheme 
#    ps8_fid, # The fiducial cosmology 
#    z_table, # Redshift steps in observation
#    dNdz, # Redshift noise in observation
#    fisher_order=[
#        'omega_b',                                    
#        'omega_cdm',                                  
#        'n_s',                                        
#        'A_s',                                        
#        'tau_reio',                                   
#        'h',                                          
#        'T_ncdm', 
#        'sigma_fog',                                   
#        'b0',                                         
#        'alpha_k2'
#    ],
#    fsky=skycover, # Sky coverage in observation
#    RSD=True, # Use RSD correction to Pm
#    FOG=True, # Use FOG correction to Pm
#    AP=True, # Use AP correction to PM
#    COV=True, #Use AP Change of Variables correction to PM
#    mu_step=mu_integral_step,
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,
#    saveplots=True,
#    savepath=ps8_convergencedir, 
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',
#                'M_ncdm',
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2'] 
#    )
#ps8_convergencetest.gen_all_plots()

# Run Fisher Forecast
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps8_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps8_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps8_fiducialset = [dict(ps8_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps8_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "EUCLID",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps8_fiducialset)]
for fidx, fcst in enumerate(ps8_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm',                                 
            'sigma_fog',                                   
            'beta0',
            'beta1',
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")

In [ ]:
dill.dump_session(os.path.join(ps8_resultsdir, 'ps8.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps8/Low_Masses/Fixed_Kmax/ps8.db')

# Save results 
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'
for fidx, fval in enumerate(ps8_forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm'],
        fisherpath=inpath+str(fidx+1)+"_T1.5K.txt"
    )
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    outdir=os.path.join(ps8_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + 'Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)

# Plot Set 9 - Relic Forecast for BOSS/eBOSS at 0.9052[K]

In [ ]:
import os                                                                       
import dill                                                                     
import shutil                                                                   
import numpy as np                                                              
import pandas as pd                                                             
import seaborn as sns                                                           
import cosmicfish as cf                                                         
import matplotlib.pyplot as plt                                                 
                                                                                
                                                                                
# Instruct pyplot to use seaborn                                                
sns.set()                                                                       
                                                                                
# Set project, data, CLASS directories                                          
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")    
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync9/")
classpath = os.path.join(projectdir, "class")                                   
                                                                                
# Specify resolution of numerical integrals                                     
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1
                                                                                
# Generate output paths                                                         
ps9_resultsdir = os.path.join(projectdir, 'results', 'ps9')       
ps9_convergencedir = os.path.join(ps9_resultsdir, 'convergence')
cf.makedirectory(ps9_resultsdir)                                        
cf.makedirectory(ps9_convergencedir)                                    
                                                                                
#Set Fiducial cosmology                                                         
# Linda Fiducial Cosmology 
ps9_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (0.9052/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "beta0" : 1.7, 
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "T_ncdm" # Fix T_ncdm or m_ncdm 
        }

# BOSS LRGs                                                               
z_table = z_table = np.array([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75])
dNdz = np.array([8., 50., 125., 222., 332., 447., 208., 30.])    
skycover = 10000. # Sky coverage of BOSS in degrees^2
                                                                                
# Demonstrate Convergence                                                       
#ps9_convergencetest = cf.convergence(                                  
#    classpath, # Path to CLASS installation                                    
#    datastore, # Path to directory holding CLASS output data                   
#    'relic', # 'relic' or 'neutrino' forecasting scheme                        
#    ps9_fid, # The fiducial cosmology                                  
#    z_table, # Redshift steps in observation                                   
#    dNdz, # Redshift noise in observation                                      
#    fisher_order=[                                                             
#        'omega_b',                                                             
#        'omega_cdm',                                                           
#        'n_s',                                                                 
#        'A_s',                                                                 
#        'tau_reio',                                                            
#        'h',                                                                   
#        'T_ncdm',                                                              
#        'sigma_fog',                                                           
#        'b0',                                                                  
#        'alpha_k2'                                                             
#    ],                                                                         
#    fcoverage_deg=skycover, # Sky coverage in observation                      
#    RSD=True, # Use RSD correction to Pm                                       
#    FOG=True, # Use FOG correction to Pm                                       
#    AP=True, # Use AP correction to PM                                         
#    COV=True, #Use AP Change of Variables correction to PM                     
#    mu_step=mu_integral_step,                                                  
#    varyfactors=[0.006, 0.007, 0.008, 0.009, 0.010], # Relative factors used to compute convergence
#    showplots=True,                                                            
#    saveplots=True,                                                            
#    savepath=ps9_convergencedir,                                       
#    plotparams= ['A_s',                                                          
#                'n_s',                                                          
#                'omega_b',                                                      
#                'omega_cdm',                                                    
#                'h',                                                            
#                'tau_reio',                                                     
#                'omega_ncdm',                                                  
#                'M_ncdm',                                                      
#                'sigmafog',                                                     
#                'b0',                                                           
#                'alphak2']                                                     
#    )                                                                          
#ps9_convergencetest.gen_all_plots()

# Run Fisher Forecast                                                           
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps9_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps9_fid['T_ncdm']*2.726 / 1.95, 3.))
ps9_fiducialset = [dict(ps9_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]})
               for midx, mval in enumerate(masses)]                             
ps9_forecastset = [cf.forecast(                                         
    classpath,                                                                  
    datastore,                                                                  
    'relic',                                                                    
    fidval,                                                                     
    z_table, 
    "EUCLID",
    dNdz,                                                                       
    fcoverage_deg=skycover,                                                     
    dstep=derivative_step,                                                      
    RSD=True,                                                                   
    FOG=True,                                                                   
    AP=True,                                                                    
    COV=True) for fididx, fidval in enumerate(ps9_fiducialset)]         
for fidx, fcst in enumerate(ps9_forecastset):                           
    fcst.gen_pm()                                                               
    fcst.gen_fisher(                                                            
        fisher_order=[                                                          
            'omega_b',                                                          
            'omega_cdm',                                                        
            'n_s',                                                              
            'A_s',                                                              
            'tau_reio',                                                         
            'h',                                                                
            'T_ncdm',                                                           
            'sigma_fog',                                                        
            'beta0',
            'beta1',
            'alpha_k2'],                                                        
        mu_step=mu_integral_step,                                               
        skipgen=False)                                                          
    print("Relic Forecast ", fidx, " complete...")                              

In [ ]:
dill.dump_session(os.path.join(ps9_resultsdir, 'ps9.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps9/Low_Masses/Fixed_Kmax/ps9.db')

# Save results                                                                  
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'
for fidx, fval in enumerate(ps9_forecastset):                     
    fval.load_cmb_fisher(                                                       
        fisher_order=[                                                          
            'omega_b',                                                          
            'omega_cdm',                                                        
            'n_s',                                                              
            'A_s',                                                              
            'tau_reio',                                                         
            'h',                                                                
            'T_ncdm'],                                                          
        fisherpath=inpath+str(fidx+1)+"_T0.9K.txt"
    ) 
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")     
    outdir=os.path.join(ps9_resultsdir, 'M'+str(fidx+1))                  
    cf.makedirectory(outdir)                                                    
    fval.export_matrices(outdir)                                                
    np.savetxt(outdir + 'Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)

# Plot Set 10 - Combine Constraints from Various Surveys

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync10/")
classpath = os.path.join(projectdir, "class")

# Combine plots
basepath = '/Users/nicholasdeporzio/Desktop/cfworkspace/results/'

# Load data
low_masses = np.geomspace(0.001, 0.1, 10)
high_masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

lownumpts = len(low_masses)
highnumpts = len(high_masses)

mattype = '/inv_fullfisher.mat'

In [ ]:
high_plotdatadir = '/Users/nicholasdeporzio/Desktop/plotdata/'
cf.makedirectory(high_plotdatadir)

np.savetxt(high_plotdatadir+'masses.csv', high_masses)

high_desi_cmbs4_1507mK_covs = [pd.read_csv(basepath+'ps4/High_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_desi_cmbs4_905mK_covs = [pd.read_csv(basepath+'ps5/High_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_euclid_cmbs4_1507mK_covs = [pd.read_csv(basepath+'ps6/High_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_euclid_cmbs4_905mK_covs = [pd.read_csv(basepath+'ps7/High_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_boss_cmbs4_1507mK_covs = [pd.read_csv(basepath+'ps8/High_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_boss_cmbs4_905mK_covs = [pd.read_csv(basepath+'ps9/High_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]

high_desi_planck_1507mK_covs = [pd.read_csv(basepath+'ps4/High_Masses/Fixed_Kmax/Planck/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_desi_planck_905mK_covs = [pd.read_csv(basepath+'ps5/High_Masses/Fixed_Kmax/Planck/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_euclid_planck_1507mK_covs = [pd.read_csv(basepath+'ps6/High_Masses/Fixed_Kmax/Planck/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_euclid_planck_905mK_covs = [pd.read_csv(basepath+'ps7/High_Masses/Fixed_Kmax/Planck/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_boss_planck_1507mK_covs = [pd.read_csv(basepath+'ps8/High_Masses/Fixed_Kmax/Planck/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_boss_planck_905mK_covs = [pd.read_csv(basepath+'ps9/High_Masses/Fixed_Kmax/Planck/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]

high_desi_cmbs4_1507mK_Terrs = [np.sqrt(high_desi_cmbs4_1507mK_covs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_desi_cmbs4_905mK_Terrs = [np.sqrt(high_desi_cmbs4_905mK_covs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_euclid_cmbs4_1507mK_Terrs = [np.sqrt(high_euclid_cmbs4_1507mK_covs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_euclid_cmbs4_905mK_Terrs = [np.sqrt(high_euclid_cmbs4_905mK_covs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_boss_cmbs4_1507mK_Terrs = [np.sqrt(high_boss_cmbs4_1507mK_covs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_boss_cmbs4_905mK_Terrs = [np.sqrt(high_boss_cmbs4_905mK_covs[idx].iloc[6,6]) for idx in range(highnumpts)]

high_desi_planck_1507mK_Terrs = [np.sqrt(high_desi_planck_1507mK_covs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_desi_planck_905mK_Terrs = [np.sqrt(high_desi_planck_905mK_covs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_euclid_planck_1507mK_Terrs = [np.sqrt(high_euclid_planck_1507mK_covs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_euclid_planck_905mK_Terrs = [np.sqrt(high_euclid_planck_905mK_covs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_boss_planck_1507mK_Terrs = [np.sqrt(high_boss_planck_1507mK_covs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_boss_planck_905mK_Terrs = [np.sqrt(high_boss_planck_905mK_covs[idx].iloc[6,6]) for idx in range(highnumpts)]

high_desi_cmbs4_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(high_desi_cmbs4_1507mK_Terrs) * np.power(1.95, -4.))
high_desi_cmbs4_905mK_Nerrs = (4. * np.power(0.9, 3.) * np.array(high_desi_cmbs4_905mK_Terrs) * np.power(1.95, -4.))
high_euclid_cmbs4_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(high_euclid_cmbs4_1507mK_Terrs) * np.power(1.95, -4.))
high_euclid_cmbs4_905mK_Nerrs = (4. * np.power(0.9, 3.) * np.array(high_euclid_cmbs4_905mK_Terrs) * np.power(1.95, -4.))
high_boss_cmbs4_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(high_boss_cmbs4_1507mK_Terrs) * np.power(1.95, -4.))
high_boss_cmbs4_905mK_Nerrs = (4. * np.power(0.9, 3.) * np.array(high_boss_cmbs4_905mK_Terrs) * np.power(1.95, -4.))

high_desi_planck_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(high_desi_planck_1507mK_Terrs) * np.power(1.95, -4.))
high_desi_planck_905mK_Nerrs = (4. * np.power(0.9, 3.) * np.array(high_desi_planck_905mK_Terrs) * np.power(1.95, -4.))
high_euclid_planck_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(high_euclid_planck_1507mK_Terrs) * np.power(1.95, -4.))
high_euclid_planck_905mK_Nerrs = (4. * np.power(0.9, 3.) * np.array(high_euclid_planck_905mK_Terrs) * np.power(1.95, -4.))
high_boss_planck_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(high_boss_planck_1507mK_Terrs) * np.power(1.95, -4.))
high_boss_planck_905mK_Nerrs = (4. * np.power(0.9, 3.) * np.array(high_boss_planck_905mK_Terrs) * np.power(1.95, -4.))

np.savetxt(high_plotdatadir+'high_cmbs4_desi_1507mK_Terrs.csv', high_desi_cmbs4_1507mK_Terrs)
np.savetxt(high_plotdatadir+'high_cmbs4_desi_905mK_Terrs.csv', high_desi_cmbs4_905mK_Terrs)
np.savetxt(high_plotdatadir+'high_cmbs4_euclid_1507mK_Terrs.csv', high_euclid_cmbs4_1507mK_Terrs) 
np.savetxt(high_plotdatadir+'high_cmbs4_euclid_905mK_Terrs.csv',  high_euclid_cmbs4_905mK_Terrs)
np.savetxt(high_plotdatadir+'high_cmbs4_boss_1507mK_Terrs.csv', high_boss_cmbs4_1507mK_Terrs)
np.savetxt(high_plotdatadir+'high_cmbs4_boss_905mK_Terrs.csv', high_boss_cmbs4_905mK_Terrs)

np.savetxt(high_plotdatadir+'high_planck_desi_1507mK_Terrs.csv', high_desi_planck_1507mK_Terrs) 
np.savetxt(high_plotdatadir+'high_planck_desi_905mK_Terrs.csv', high_desi_planck_905mK_Terrs)
np.savetxt(high_plotdatadir+'high_planck_euclid_1507mK_Terrs.csv', high_euclid_planck_1507mK_Terrs)
np.savetxt(high_plotdatadir+'high_planck_euclid_905mK_Terrs.csv', high_euclid_planck_905mK_Terrs)
np.savetxt(high_plotdatadir+'high_planck_boss_1507mK_Terrs.csv', high_boss_planck_1507mK_Terrs)
np.savetxt(high_plotdatadir+'high_planck_boss_905mK_Terrs.csv', high_boss_planck_905mK_Terrs)

np.savetxt(high_plotdatadir+'high_cmbs4_desi_1507mK_Nerrs.csv', high_desi_cmbs4_1507mK_Nerrs)
np.savetxt(high_plotdatadir+'high_cmbs4_desi_905mK_Nerrs.csv', high_desi_cmbs4_905mK_Nerrs)
np.savetxt(high_plotdatadir+'high_cmbs4_euclid_1507mK_Nerrs.csv', high_euclid_cmbs4_1507mK_Nerrs) 
np.savetxt(high_plotdatadir+'high_cmbs4_euclid_905mK_Nerrs.csv',  high_euclid_cmbs4_905mK_Nerrs)
np.savetxt(high_plotdatadir+'high_cmbs4_boss_1507mK_Nerrs.csv', high_boss_cmbs4_1507mK_Nerrs)
np.savetxt(high_plotdatadir+'high_cmbs4_boss_905mK_Nerrs.csv', high_boss_cmbs4_905mK_Nerrs)

np.savetxt(high_plotdatadir+'high_planck_desi_1507mK_Nerrs.csv', high_desi_planck_1507mK_Nerrs) 
np.savetxt(high_plotdatadir+'high_planck_desi_905mK_Nerrs.csv', high_desi_planck_905mK_Nerrs)
np.savetxt(high_plotdatadir+'high_planck_euclid_1507mK_Nerrs.csv', high_euclid_planck_1507mK_Nerrs)
np.savetxt(high_plotdatadir+'high_planck_euclid_905mK_Nerrs.csv', high_euclid_planck_905mK_Nerrs)
np.savetxt(high_plotdatadir+'high_planck_boss_1507mK_Nerrs.csv', high_boss_planck_1507mK_Nerrs)
np.savetxt(high_plotdatadir+'high_planck_boss_905mK_Nerrs.csv', high_boss_planck_905mK_Nerrs)

In [ ]:
high_desi_cmbs4_1507mK_omegacovs = [pd.read_csv(basepath+'ps4/High_Masses/Fixed_Kmax/CMBS4/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_desi_cmbs4_905mK_omegacovs = [pd.read_csv(basepath+'ps5/High_Masses/Fixed_Kmax/CMBS4/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_euclid_cmbs4_1507mK_omegacovs = [pd.read_csv(basepath+'ps6/High_Masses/Fixed_Kmax/CMBS4/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_euclid_cmbs4_905mK_omegacovs = [pd.read_csv(basepath+'ps7/High_Masses/Fixed_Kmax/CMBS4/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_boss_cmbs4_1507mK_omegacovs = [pd.read_csv(basepath+'ps8/High_Masses/Fixed_Kmax/CMBS4/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_boss_cmbs4_905mK_omegacovs = [pd.read_csv(basepath+'ps9/High_Masses/Fixed_Kmax/CMBS4/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]

high_desi_planck_1507mK_omegacovs = [pd.read_csv(basepath+'ps4/High_Masses/Fixed_Kmax/Planck/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_desi_planck_905mK_omegacovs = [pd.read_csv(basepath+'ps5/High_Masses/Fixed_Kmax/Planck/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_euclid_planck_1507mK_omegacovs = [pd.read_csv(basepath+'ps6/High_Masses/Fixed_Kmax/Planck/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_euclid_planck_905mK_omegacovs = [pd.read_csv(basepath+'ps7/High_Masses/Fixed_Kmax/Planck/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_boss_planck_1507mK_omegacovs = [pd.read_csv(basepath+'ps8/High_Masses/Fixed_Kmax/Planck/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]
high_boss_planck_905mK_omegacovs = [pd.read_csv(basepath+'ps9/High_Masses/Fixed_Kmax/Planck/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(highnumpts)]

high_desi_cmbs4_1507mK_omegaerrs = [np.sqrt(high_desi_cmbs4_1507mK_omegacovs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_desi_cmbs4_905mK_omegaerrs = [np.sqrt(high_desi_cmbs4_905mK_omegacovs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_euclid_cmbs4_1507mK_omegaerrs = [np.sqrt(high_euclid_cmbs4_1507mK_omegacovs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_euclid_cmbs4_905mK_omegaerrs = [np.sqrt(high_euclid_cmbs4_905mK_omegacovs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_boss_cmbs4_1507mK_omegaerrs = [np.sqrt(high_boss_cmbs4_1507mK_omegacovs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_boss_cmbs4_905mK_omegaerrs = [np.sqrt(high_boss_cmbs4_905mK_omegacovs[idx].iloc[6,6]) for idx in range(highnumpts)]

high_desi_planck_1507mK_omegaerrs = [np.sqrt(high_desi_planck_1507mK_omegacovs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_desi_planck_905mK_omegaerrs = [np.sqrt(high_desi_planck_905mK_omegacovs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_euclid_planck_1507mK_omegaerrs = [np.sqrt(high_euclid_planck_1507mK_omegacovs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_euclid_planck_905mK_omegaerrs = [np.sqrt(high_euclid_planck_905mK_omegacovs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_boss_planck_1507mK_omegaerrs = [np.sqrt(high_boss_planck_1507mK_omegacovs[idx].iloc[6,6]) for idx in range(highnumpts)]
high_boss_planck_905mK_omegaerrs = [np.sqrt(high_boss_planck_905mK_omegacovs[idx].iloc[6,6]) for idx in range(highnumpts)]

np.savetxt(high_plotdatadir+'high_cmbs4_desi_1507mK_omegaerrs.csv', high_desi_cmbs4_1507mK_omegaerrs)
np.savetxt(high_plotdatadir+'high_cmbs4_desi_905mK_omegaerrs.csv', high_desi_cmbs4_905mK_omegaerrs)
np.savetxt(high_plotdatadir+'high_cmbs4_euclid_1507mK_omegaerrs.csv', high_euclid_cmbs4_1507mK_omegaerrs) 
np.savetxt(high_plotdatadir+'high_cmbs4_euclid_905mK_omegaerrs.csv',  high_euclid_cmbs4_905mK_omegaerrs)
np.savetxt(high_plotdatadir+'high_cmbs4_boss_1507mK_omegaerrs.csv', high_boss_cmbs4_1507mK_omegaerrs)
np.savetxt(high_plotdatadir+'high_cmbs4_boss_905mK_omegaerrs.csv', high_boss_cmbs4_905mK_omegaerrs)

np.savetxt(high_plotdatadir+'high_planck_desi_1507mK_omegaerrs.csv', high_desi_planck_1507mK_omegaerrs) 
np.savetxt(high_plotdatadir+'high_planck_desi_905mK_omegaerrs.csv', high_desi_planck_905mK_omegaerrs)
np.savetxt(high_plotdatadir+'high_planck_euclid_1507mK_omegaerrs.csv', high_euclid_planck_1507mK_omegaerrs)
np.savetxt(high_plotdatadir+'high_planck_euclid_905mK_omegaerrs.csv', high_euclid_planck_905mK_omegaerrs)
np.savetxt(high_plotdatadir+'high_planck_boss_1507mK_omegaerrs.csv', high_boss_planck_1507mK_omegaerrs)
np.savetxt(high_plotdatadir+'high_planck_boss_905mK_omegaerrs.csv', high_boss_planck_905mK_omegaerrs)


In [ ]:
low_plotdatadir = '/Users/nicholasdeporzio/Desktop/plotdata/'
cf.makedirectory(low_plotdatadir)

np.savetxt(low_plotdatadir+'masses.csv', low_masses)

low_desi_cmbs4_1507mK_covs = [pd.read_csv(basepath+'ps4/Low_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_desi_cmbs4_905mK_covs = [pd.read_csv(basepath+'ps5/Low_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_euclid_cmbs4_1507mK_covs = [pd.read_csv(basepath+'ps6/Low_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_euclid_cmbs4_905mK_covs = [pd.read_csv(basepath+'ps7/Low_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_boss_cmbs4_1507mK_covs = [pd.read_csv(basepath+'ps8/Low_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_boss_cmbs4_905mK_covs = [pd.read_csv(basepath+'ps9/Low_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]

low_desi_planck_1507mK_covs = [pd.read_csv(basepath+'ps4/Low_Masses/Fixed_Kmax/Planck/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_desi_planck_905mK_covs = [pd.read_csv(basepath+'ps5/Low_Masses/Fixed_Kmax/Planck/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_euclid_planck_1507mK_covs = [pd.read_csv(basepath+'ps6/Low_Masses/Fixed_Kmax/Planck/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_euclid_planck_905mK_covs = [pd.read_csv(basepath+'ps7/Low_Masses/Fixed_Kmax/Planck/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_boss_planck_1507mK_covs = [pd.read_csv(basepath+'ps8/Low_Masses/Fixed_Kmax/Planck/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_boss_planck_905mK_covs = [pd.read_csv(basepath+'ps9/Low_Masses/Fixed_Kmax/Planck/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]

low_desi_cmbs4_1507mK_Terrs = [np.sqrt(low_desi_cmbs4_1507mK_covs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_desi_cmbs4_905mK_Terrs = [np.sqrt(low_desi_cmbs4_905mK_covs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_euclid_cmbs4_1507mK_Terrs = [np.sqrt(low_euclid_cmbs4_1507mK_covs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_euclid_cmbs4_905mK_Terrs = [np.sqrt(low_euclid_cmbs4_905mK_covs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_boss_cmbs4_1507mK_Terrs = [np.sqrt(low_boss_cmbs4_1507mK_covs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_boss_cmbs4_905mK_Terrs = [np.sqrt(low_boss_cmbs4_905mK_covs[idx].iloc[6,6]) for idx in range(lownumpts)]

low_desi_planck_1507mK_Terrs = [np.sqrt(low_desi_planck_1507mK_covs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_desi_planck_905mK_Terrs = [np.sqrt(low_desi_planck_905mK_covs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_euclid_planck_1507mK_Terrs = [np.sqrt(low_euclid_planck_1507mK_covs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_euclid_planck_905mK_Terrs = [np.sqrt(low_euclid_planck_905mK_covs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_boss_planck_1507mK_Terrs = [np.sqrt(low_boss_planck_1507mK_covs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_boss_planck_905mK_Terrs = [np.sqrt(low_boss_planck_905mK_covs[idx].iloc[6,6]) for idx in range(lownumpts)]

low_desi_cmbs4_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(low_desi_cmbs4_1507mK_Terrs) * np.power(1.95, -4.))
low_desi_cmbs4_905mK_Nerrs = (4. * np.power(0.9, 3.) * np.array(low_desi_cmbs4_905mK_Terrs) * np.power(1.95, -4.))
low_euclid_cmbs4_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(low_euclid_cmbs4_1507mK_Terrs) * np.power(1.95, -4.))
low_euclid_cmbs4_905mK_Nerrs = (4. * np.power(0.9, 3.) * np.array(low_euclid_cmbs4_905mK_Terrs) * np.power(1.95, -4.))
low_boss_cmbs4_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(low_boss_cmbs4_1507mK_Terrs) * np.power(1.95, -4.))
low_boss_cmbs4_905mK_Nerrs = (4. * np.power(0.9, 3.) * np.array(low_boss_cmbs4_905mK_Terrs) * np.power(1.95, -4.))

low_desi_planck_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(low_desi_planck_1507mK_Terrs) * np.power(1.95, -4.))
low_desi_planck_905mK_Nerrs = (4. * np.power(0.9, 3.) * np.array(low_desi_planck_905mK_Terrs) * np.power(1.95, -4.))
low_euclid_planck_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(low_euclid_planck_1507mK_Terrs) * np.power(1.95, -4.))
low_euclid_planck_905mK_Nerrs = (4. * np.power(0.9, 3.) * np.array(low_euclid_planck_905mK_Terrs) * np.power(1.95, -4.))
low_boss_planck_1507mK_Nerrs = (4. * np.power(1.5, 3.) * np.array(low_boss_planck_1507mK_Terrs) * np.power(1.95, -4.))
low_boss_planck_905mK_Nerrs = (4. * np.power(0.9, 3.) * np.array(low_boss_planck_905mK_Terrs) * np.power(1.95, -4.))

np.savetxt(low_plotdatadir+'low_cmbs4_desi_1507mK_Terrs.csv', low_desi_cmbs4_1507mK_Terrs)
np.savetxt(low_plotdatadir+'low_cmbs4_desi_905mK_Terrs.csv', low_desi_cmbs4_905mK_Terrs)
np.savetxt(low_plotdatadir+'low_cmbs4_euclid_1507mK_Terrs.csv', low_euclid_cmbs4_1507mK_Terrs) 
np.savetxt(low_plotdatadir+'low_cmbs4_euclid_905mK_Terrs.csv',  low_euclid_cmbs4_905mK_Terrs)
np.savetxt(low_plotdatadir+'low_cmbs4_boss_1507mK_Terrs.csv', low_boss_cmbs4_1507mK_Terrs)
np.savetxt(low_plotdatadir+'low_cmbs4_boss_905mK_Terrs.csv', low_boss_cmbs4_905mK_Terrs)

np.savetxt(low_plotdatadir+'low_planck_desi_1507mK_Terrs.csv', low_desi_planck_1507mK_Terrs) 
np.savetxt(low_plotdatadir+'low_planck_desi_905mK_Terrs.csv', low_desi_planck_905mK_Terrs)
np.savetxt(low_plotdatadir+'low_planck_euclid_1507mK_Terrs.csv', low_euclid_planck_1507mK_Terrs)
np.savetxt(low_plotdatadir+'low_planck_euclid_905mK_Terrs.csv', low_euclid_planck_905mK_Terrs)
np.savetxt(low_plotdatadir+'low_planck_boss_1507mK_Terrs.csv', low_boss_planck_1507mK_Terrs)
np.savetxt(low_plotdatadir+'low_planck_boss_905mK_Terrs.csv', low_boss_planck_905mK_Terrs)

np.savetxt(low_plotdatadir+'low_cmbs4_desi_1507mK_Nerrs.csv', low_desi_cmbs4_1507mK_Nerrs)
np.savetxt(low_plotdatadir+'low_cmbs4_desi_905mK_Nerrs.csv', low_desi_cmbs4_905mK_Nerrs)
np.savetxt(low_plotdatadir+'low_cmbs4_euclid_1507mK_Nerrs.csv', low_euclid_cmbs4_1507mK_Nerrs) 
np.savetxt(low_plotdatadir+'low_cmbs4_euclid_905mK_Nerrs.csv',  low_euclid_cmbs4_905mK_Nerrs)
np.savetxt(low_plotdatadir+'low_cmbs4_boss_1507mK_Nerrs.csv', low_boss_cmbs4_1507mK_Nerrs)
np.savetxt(low_plotdatadir+'low_cmbs4_boss_905mK_Nerrs.csv', low_boss_cmbs4_905mK_Nerrs)

np.savetxt(low_plotdatadir+'low_planck_desi_1507mK_Nerrs.csv', low_desi_planck_1507mK_Nerrs) 
np.savetxt(low_plotdatadir+'low_planck_desi_905mK_Nerrs.csv', low_desi_planck_905mK_Nerrs)
np.savetxt(low_plotdatadir+'low_planck_euclid_1507mK_Nerrs.csv', low_euclid_planck_1507mK_Nerrs)
np.savetxt(low_plotdatadir+'low_planck_euclid_905mK_Nerrs.csv', low_euclid_planck_905mK_Nerrs)
np.savetxt(low_plotdatadir+'low_planck_boss_1507mK_Nerrs.csv', low_boss_planck_1507mK_Nerrs)
np.savetxt(low_plotdatadir+'low_planck_boss_905mK_Nerrs.csv', low_boss_planck_905mK_Nerrs)

In [ ]:
low_desi_cmbs4_1507mK_omegacovs = [pd.read_csv(basepath+'ps4/Low_Masses/Fixed_Kmax/CMBS4/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_desi_cmbs4_905mK_omegacovs = [pd.read_csv(basepath+'ps5/Low_Masses/Fixed_Kmax/CMBS4/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_euclid_cmbs4_1507mK_omegacovs = [pd.read_csv(basepath+'ps6/Low_Masses/Fixed_Kmax/CMBS4/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_euclid_cmbs4_905mK_omegacovs = [pd.read_csv(basepath+'ps7/Low_Masses/Fixed_Kmax/CMBS4/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_boss_cmbs4_1507mK_omegacovs = [pd.read_csv(basepath+'ps8/Low_Masses/Fixed_Kmax/CMBS4/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_boss_cmbs4_905mK_omegacovs = [pd.read_csv(basepath+'ps9/Low_Masses/Fixed_Kmax/CMBS4/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]

low_desi_planck_1507mK_omegacovs = [pd.read_csv(basepath+'ps4/Low_Masses/Fixed_Kmax/Planck/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_desi_planck_905mK_omegacovs = [pd.read_csv(basepath+'ps5/Low_Masses/Fixed_Kmax/Planck/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_euclid_planck_1507mK_omegacovs = [pd.read_csv(basepath+'ps6/Low_Masses/Fixed_Kmax/Planck/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_euclid_planck_905mK_omegacovs = [pd.read_csv(basepath+'ps7/Low_Masses/Fixed_Kmax/Planck/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_boss_planck_1507mK_omegacovs = [pd.read_csv(basepath+'ps8/Low_Masses/Fixed_Kmax/Planck/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]
low_boss_planck_905mK_omegacovs = [pd.read_csv(basepath+'ps9/Low_Masses/Fixed_Kmax/Planck/omega_version/M'+str(idx+1)+mattype, sep='\t', header=0) for idx in range(lownumpts)]

low_desi_cmbs4_1507mK_omegaerrs = [np.sqrt(low_desi_cmbs4_1507mK_omegacovs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_desi_cmbs4_905mK_omegaerrs = [np.sqrt(low_desi_cmbs4_905mK_omegacovs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_euclid_cmbs4_1507mK_omegaerrs = [np.sqrt(low_euclid_cmbs4_1507mK_omegacovs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_euclid_cmbs4_905mK_omegaerrs = [np.sqrt(low_euclid_cmbs4_905mK_omegacovs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_boss_cmbs4_1507mK_omegaerrs = [np.sqrt(low_boss_cmbs4_1507mK_omegacovs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_boss_cmbs4_905mK_omegaerrs = [np.sqrt(low_boss_cmbs4_905mK_omegacovs[idx].iloc[6,6]) for idx in range(lownumpts)]

low_desi_planck_1507mK_omegaerrs = [np.sqrt(low_desi_planck_1507mK_omegacovs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_desi_planck_905mK_omegaerrs = [np.sqrt(low_desi_planck_905mK_omegacovs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_euclid_planck_1507mK_omegaerrs = [np.sqrt(low_euclid_planck_1507mK_omegacovs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_euclid_planck_905mK_omegaerrs = [np.sqrt(low_euclid_planck_905mK_omegacovs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_boss_planck_1507mK_omegaerrs = [np.sqrt(low_boss_planck_1507mK_omegacovs[idx].iloc[6,6]) for idx in range(lownumpts)]
low_boss_planck_905mK_omegaerrs = [np.sqrt(low_boss_planck_905mK_omegacovs[idx].iloc[6,6]) for idx in range(lownumpts)]

np.savetxt(low_plotdatadir+'low_cmbs4_desi_1507mK_omegaerrs.csv', low_desi_cmbs4_1507mK_omegaerrs)
np.savetxt(low_plotdatadir+'low_cmbs4_desi_905mK_omegaerrs.csv', low_desi_cmbs4_905mK_omegaerrs)
np.savetxt(low_plotdatadir+'low_cmbs4_euclid_1507mK_omegaerrs.csv', low_euclid_cmbs4_1507mK_omegaerrs) 
np.savetxt(low_plotdatadir+'low_cmbs4_euclid_905mK_omegaerrs.csv',  low_euclid_cmbs4_905mK_omegaerrs)
np.savetxt(low_plotdatadir+'low_cmbs4_boss_1507mK_omegaerrs.csv', low_boss_cmbs4_1507mK_omegaerrs)
np.savetxt(low_plotdatadir+'low_cmbs4_boss_905mK_omegaerrs.csv', low_boss_cmbs4_905mK_omegaerrs)

np.savetxt(low_plotdatadir+'low_planck_desi_1507mK_omegaerrs.csv', low_desi_planck_1507mK_omegaerrs) 
np.savetxt(low_plotdatadir+'low_planck_desi_905mK_omegaerrs.csv', low_desi_planck_905mK_omegaerrs)
np.savetxt(low_plotdatadir+'low_planck_euclid_1507mK_omegaerrs.csv', low_euclid_planck_1507mK_omegaerrs)
np.savetxt(low_plotdatadir+'low_planck_euclid_905mK_omegaerrs.csv', low_euclid_planck_905mK_omegaerrs)
np.savetxt(low_plotdatadir+'low_planck_boss_1507mK_omegaerrs.csv', low_boss_planck_1507mK_omegaerrs)
np.savetxt(low_plotdatadir+'low_planck_boss_905mK_omegaerrs.csv', low_boss_planck_905mK_omegaerrs)

In [ ]:
#butt
from scipy.integrate import quad
def integrand(p, m, T):
    return (2./np.power(2 * np.pi, 3) * (1/ (np.exp(p/T) + 1)) 
            * np.sqrt(np.power(p, 2) + np.power(m, 2))  * (4 * np.pi * np.power(p, 2)))

omegashot = np.array([quad(integrand, 0, 10, args=(mval, 1.5e-4 ))[0] for mval in high_masses])
omegascold = np.array([quad(integrand, 0, 10, args=(mval, 0.9e-4 ))[0] for mval in high_masses])

omegascmb_hot = np.array([quad(integrand, 0, 10, args=(mval, 1.5*1100*1e-4 ))[0] for mval in high_masses])
omegascmb_cold = np.array([quad(integrand, 0, 10, args=(mval, 0.9*1100*1e-4 ))[0] for mval in high_masses])

relomegaerrhot = omegascmb_hot/omegashot
relomegaerrcold = omegascmb_cold/omegascold

dstepuphot = np.array([quad(integrand, 0, 10, args=(mval, 1.01*1.5*1e-4 ))[0] for mval in high_masses])
dstepdownhot = np.array([quad(integrand, 0, 10, args=(mval, 0.99*1.5*1e-4 ))[0] for mval in high_masses])
dstepupcold = np.array([quad(integrand, 0, 10, args=(mval, 1.01*0.9*1e-4 ))[0] for mval in high_masses])
dstepdowncold = np.array([quad(integrand, 0, 10, args=(mval, 0.99*0.9*1e-4 ))[0] for mval in high_masses])

dstepupcmbhot = np.array([quad(integrand, 0, 10, args=(mval, 1.001*1.5*1100*1e-4 ))[0] for mval in high_masses])
dstepdowncmbhot = np.array([quad(integrand, 0, 10, args=(mval, 0.999*1.5*1100*1e-4 ))[0] for mval in high_masses])
dstepupcmbcold = np.array([quad(integrand, 0, 10, args=(mval, 1.001*0.9*1100*1e-4 ))[0] for mval in high_masses])
dstepdowncmbcold = np.array([quad(integrand, 0, 10, args=(mval, 0.999*0.9*1100*1e-4 ))[0] for mval in high_masses])

domegadThot = (dstepuphot-dstepdownhot)/(2*0.01*1.5*1e-4)
domegadTcold = (dstepupcold-dstepdowncold)/(2*0.01*0.9*1e-4)

domegacmbdThot = (dstepupcmbhot-dstepdowncmbhot)/(2*0.001*1.5*1e-4)
domegacmbdTcold = (dstepupcmbcold-dstepdowncmbcold)/(2*0.001*0.9*1e-4)

dlogomegadlogT_hot = (1.5*1e-4) * domegadThot / omegashot 
dlogomegadlogT_cold = (0.9*1e-4) * domegadTcold / omegascold

dlogomegacmbdlogT_hot = (1.5*1e-4) * domegacmbdThot / omegascmb_hot 
dlogomegacmbdlogT_cold = (0.9*1e-4) * domegacmbdTcold / omegascmb_cold

deltaTdomegadT_hot = dlogomegadlogT_hot * omegashot * (1/(1.5*1e-4)) * high_euclid_cmbs4_1507mK_Terrs
deltaTdomegadT_cold = dlogomegadlogT_cold * omegascold * (1/(0.9*1e-4)) * high_euclid_cmbs4_905mK_Terrs


In [ ]:
plotdatadir = '/Users/nicholasdeporzio/Desktop/plotdata/'
cf.makedirectory(plotdatadir)
######################################

plt.figure(figsize=(15,7.5))
plt.semilogx(high_masses, dlogomegacmbdlogT_hot, label="$T_{\chi, 0} = 1.5 K$")
plt.semilogx(high_masses, dlogomegacmbdlogT_cold, label="$T_{\chi, 0} = 0.9 K$")
#plt.title(r"$T_{\chi, 0}$ = 1.5 K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$dlog\omega_\chi / dlogT_\chi$", fontsize=24)
plt.legend(fontsize=18)
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/dlogplot.png")
plt.show()

######################################
plt.figure(figsize=(15,7.5))
plt.semilogx(high_masses, np.array(high_boss_cmbs4_1507mK_Terrs) , label="CMBS4 + BOSS", linestyle='dashed', color='red')
plt.semilogx(high_masses, np.array(high_desi_cmbs4_1507mK_Terrs), label="CMBS4 + DESI", linestyle='dashed', color='green')
plt.semilogx(high_masses, np.array(high_euclid_cmbs4_1507mK_Terrs) , label="CMBS4 + EUCLID", linestyle='dashed', color='blue')
plt.semilogx(high_masses, np.array(high_boss_planck_1507mK_Terrs) , label="Planck + BOSS", linestyle='solid', color='red')
plt.semilogx(high_masses, np.array(high_desi_planck_1507mK_Terrs), label="Planck + DESI", linestyle='solid', color='green')
plt.semilogx(high_masses, np.array(high_euclid_planck_1507mK_Terrs) , label="Planck + EUCLID", linestyle='solid', color='blue')

#plt.semilogx(low_masses, np.array(low_boss_cmbs4_1507mK_Terrs) , linestyle='dashed', color='red')
#plt.semilogx(low_masses, np.array(low_desi_cmbs4_1507mK_Terrs), linestyle='dashed', color='green')
#plt.semilogx(low_masses, np.array(low_euclid_cmbs4_1507mK_Terrs) , linestyle='dashed', color='blue')
#plt.semilogx(low_masses, np.array(low_boss_planck_1507mK_Terrs) , linestyle='solid', color='red')
#plt.semilogx(low_masses, np.array(low_desi_planck_1507mK_Terrs), linestyle='solid', color='green')
#plt.semilogx(low_masses, np.array(low_euclid_planck_1507mK_Terrs) , linestyle='solid', color='blue')

#sigmaline1 = (lownumpts + highnumpts) * [1.5072/8]
#sigmaline1 = (highnumpts) * [1.5072/8]

#sigmaline1 = omegascmb_hot / 3.
#plt.semilogx(np.append(low_masses, high_masses), sigmaline1, label=r"$2\sigma$", linestyle='solid', color='black')
#plt.semilogx(high_masses, sigmaline1, label=r"$2\sigma$", linestyle='solid', color='black')
#np.savetxt(plotdatadir+'1507mK_T_5sigma.csv', [np.append(low_masses, high_masses), sigmaline1])

plt.title(r"$T_{\chi, 0}$ = 1.5 K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{T_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='center right')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig(plotdatadir + "relicforecast_T1507mk_Terr.png")
plt.show()

##########################################

##
cutmasses=np.delete(high_masses, 3)
cut905mKTerrboss = np.delete(high_boss_planck_905mK_Terrs, 3)
cut905mKTerrdesi = np.delete(high_desi_planck_905mK_Terrs, 3)
cut905mKTerreuclid = np.delete(high_euclid_planck_905mK_Terrs, 3)
##


plt.figure(figsize=(15,7.5))

plt.semilogx(high_masses, np.array(high_boss_cmbs4_905mK_Terrs) , label="CMBS4 + BOSS", linestyle='dashed', color='red')
plt.semilogx(high_masses, np.array(high_desi_cmbs4_905mK_Terrs), label="CMBS4 + DESI", linestyle='dashed', color='green')
plt.semilogx(high_masses, np.array(high_euclid_cmbs4_905mK_Terrs) , label="CMBS4 + EUCLID", linestyle='dashed', color='blue')
plt.semilogx(cutmasses, np.array(cut905mKTerrboss) , label="Planck + BOSS", linestyle='solid', color='red')
plt.semilogx(cutmasses, np.array(cut905mKTerrdesi), label="Planck + DESI", linestyle='solid', color='green')
plt.semilogx(cutmasses, np.array(cut905mKTerreuclid) , label="Planck + EUCLID", linestyle='solid', color='blue')

#plt.semilogx(low_masses, np.array(low_boss_cmbs4_905mK_Terrs) , linestyle='dashed', color='red')
#plt.semilogx(low_masses, np.array(low_desi_cmbs4_905mK_Terrs), linestyle='dashed', color='green')
#plt.semilogx(low_masses, np.array(low_euclid_cmbs4_905mK_Terrs) , linestyle='dashed', color='blue')
#plt.semilogx(low_masses, np.array(low_boss_planck_905mK_Terrs) , linestyle='solid', color='red')
#plt.semilogx(low_masses, np.array(low_desi_planck_905mK_Terrs), linestyle='solid', color='green')
#plt.semilogx(low_masses, np.array(low_euclid_planck_905mK_Terrs) , linestyle='solid', color='blue')

#sigmaline2 = (lownumpts + highnumpts) * [0.9052/8]
#sigmaline2 = (highnumpts) * [0.9052/8]

#sigmaline2 = omegascmb_cold / 3.

#plt.semilogx(np.append(low_masses, high_masses), sigmaline2, label=r"$5\sigma$", linestyle='solid', color='black')
#plt.semilogx(high_masses, sigmaline2, label=r"$2\sigma ~N_{eff}$", linestyle='solid', color='black')
#np.savetxt(plotdatadir+'905mK_T_5sigma.csv', [np.append(low_masses, high_masses), sigmaline2])

plt.title(r"$T_{\chi, 0}$ = 0.9 K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{T_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='upper right')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig(plotdatadir+"relicforecast_T905mk_Terr.png")
plt.show()


#####################
plt.figure(figsize=(15,7.5))
plt.semilogx(high_masses, np.array(high_boss_cmbs4_1507mK_Nerrs) , label="CMBS4 + BOSS", linestyle='dashed', color='red')
plt.semilogx(high_masses, np.array(high_desi_cmbs4_1507mK_Nerrs), label="CMBS4 + DESI", linestyle='dashed', color='green')
plt.semilogx(high_masses, np.array(high_euclid_cmbs4_1507mK_Nerrs) , label="CMBS4 + EUCLID", linestyle='dashed', color='blue')
plt.semilogx(high_masses, np.array(high_boss_planck_1507mK_Nerrs) , label="Planck + BOSS", linestyle='solid', color='red')
plt.semilogx(high_masses, np.array(high_desi_planck_1507mK_Nerrs), label="Planck + DESI", linestyle='solid', color='green')
plt.semilogx(high_masses, np.array(high_euclid_planck_1507mK_Nerrs) , label="Planck + EUCLID", linestyle='solid', color='blue')

plt.semilogx(low_masses, np.array(low_boss_cmbs4_1507mK_Nerrs) , linestyle='dashed', color='red')
plt.semilogx(low_masses, np.array(low_desi_cmbs4_1507mK_Nerrs), linestyle='dashed', color='green')
plt.semilogx(low_masses, np.array(low_euclid_cmbs4_1507mK_Nerrs) , linestyle='dashed', color='blue')
plt.semilogx(low_masses, np.array(low_boss_planck_1507mK_Nerrs) , linestyle='solid', color='red')
plt.semilogx(low_masses, np.array(low_desi_planck_1507mK_Nerrs), linestyle='solid', color='green')
plt.semilogx(low_masses, np.array(low_euclid_planck_1507mK_Nerrs) , linestyle='solid', color='blue')

sigmaline3 = (lownumpts + highnumpts) * [4. * np.power(1.5072, 3) * np.power(1.95, -4) * (1.5072/5)]
plt.semilogx(np.append(low_masses, high_masses), sigmaline3, label=r"$5\sigma$", linestyle='solid', color='black')
np.savetxt(plotdatadir+'1507mK_Neff_5sigma.csv', [np.append(low_masses, high_masses), sigmaline3])

plt.title(r"$T_{\chi, 0}$ = 1.5 K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{Neff}$", fontsize=24)
plt.legend(fontsize=18, loc='center right')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig(plotdatadir+"relicforecast_T1507mk_Nerr.png")
plt.show()

#####################
plt.figure(figsize=(15,7.5))
plt.semilogx(high_masses, np.array(high_boss_cmbs4_905mK_Nerrs) , label="CMBS4 + BOSS", linestyle='dashed', color='red')
plt.semilogx(high_masses, np.array(high_desi_cmbs4_905mK_Nerrs), label="CMBS4 + DESI", linestyle='dashed', color='green')
plt.semilogx(high_masses, np.array(high_euclid_cmbs4_905mK_Nerrs) , label="CMBS4 + EUCLID", linestyle='dashed', color='blue')
plt.semilogx(high_masses, np.array(high_boss_planck_905mK_Nerrs) , label="Planck + BOSS", linestyle='solid', color='red')
plt.semilogx(high_masses, np.array(high_desi_planck_905mK_Nerrs), label="Planck + DESI", linestyle='solid', color='green')
plt.semilogx(high_masses, np.array(high_euclid_planck_905mK_Nerrs) , label="Planck + EUCLID", linestyle='solid', color='blue')

plt.semilogx(low_masses, np.array(low_boss_cmbs4_905mK_Nerrs) , linestyle='dashed', color='red')
plt.semilogx(low_masses, np.array(low_desi_cmbs4_905mK_Nerrs), linestyle='dashed', color='green')
plt.semilogx(low_masses, np.array(low_euclid_cmbs4_905mK_Nerrs) , linestyle='dashed', color='blue')
plt.semilogx(low_masses, np.array(low_boss_planck_905mK_Nerrs) , linestyle='solid', color='red')
plt.semilogx(low_masses, np.array(low_desi_planck_905mK_Nerrs), linestyle='solid', color='green')
plt.semilogx(low_masses, np.array(low_euclid_planck_905mK_Nerrs) , linestyle='solid', color='blue')

sigmaline4 = (lownumpts + highnumpts) * [4. * np.power(0.9052, 3) * np.power(1.95, -4) * (0.9052/5)]
plt.semilogx(np.append(low_masses, high_masses), sigmaline4, label=r"$5\sigma$", linestyle='solid', color='black')
np.savetxt(plotdatadir+'905mK_Neff_5sigma.csv', [np.append(low_masses, high_masses), sigmaline4])

plt.title(r"$T_{\chi, 0}$ = 0.9 K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{Neff}$", fontsize=24)
plt.legend(fontsize=18, loc='upper right')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig(plotdatadir+"relicforecast_T905mk_Nerr.png")
plt.show()

#####################
plt.figure(figsize=(15,7.5))
#plt.semilogx(high_masses, np.array(high_boss_cmbs4_1507mK_omegaerrs) , label="CMBS4 + BOSS", linestyle='dashed', color='red')
#plt.semilogx(high_masses, np.array(high_desi_cmbs4_1507mK_omegaerrs), label="CMBS4 + DESI", linestyle='dashed', color='green')
#plt.semilogx(high_masses, np.array(high_euclid_cmbs4_1507mK_omegaerrs) , label="CMBS4 + EUCLID", linestyle='dashed', color='blue')
#plt.semilogx(high_masses, np.array(high_boss_planck_1507mK_omegaerrs) , label="Planck + BOSS", linestyle='solid', color='red')
#plt.semilogx(high_masses, np.array(high_desi_planck_1507mK_omegaerrs), label="Planck + DESI", linestyle='solid', color='green')
#plt.semilogx(high_masses, np.array(high_euclid_planck_1507mK_omegaerrs) , label="Planck + EUCLID", linestyle='solid', color='blue')

#plt.semilogx(low_masses, np.array(low_boss_cmbs4_1507mK_omegaerrs) , linestyle='dashed', color='red')
#plt.semilogx(low_masses, np.array(low_desi_cmbs4_1507mK_omegaerrs), linestyle='dashed', color='green')
#plt.semilogx(low_masses, np.array(low_euclid_cmbs4_1507mK_omegaerrs) , linestyle='dashed', color='blue')
#plt.semilogx(low_masses, np.array(low_boss_planck_1507mK_omegaerrs) , linestyle='solid', color='red')
#plt.semilogx(low_masses, np.array(low_desi_planck_1507mK_omegaerrs), linestyle='solid', color='green')
#plt.semilogx(low_masses, np.array(low_euclid_planck_1507mK_omegaerrs) , linestyle='solid', color='blue')





plt.loglog(high_masses, domegacmbdThot * np.array(high_boss_cmbs4_1507mK_Terrs) * 1e-4 , label="CMBS4 + BOSS", linestyle='dashed', color='red')
plt.loglog(high_masses, domegacmbdThot * np.array(high_desi_cmbs4_1507mK_Terrs) * 1e-4, label="CMBS4 + DESI", linestyle='dashed', color='green')
plt.loglog(high_masses, domegacmbdThot * np.array(high_euclid_cmbs4_1507mK_Terrs) * 1e-4 , label="CMBS4 + EUCLID", linestyle='dashed', color='blue')
plt.loglog(high_masses, domegacmbdThot * np.array(high_boss_planck_1507mK_Terrs) * 1e-4 , label="Planck + BOSS", linestyle='solid', color='red')
plt.loglog(high_masses, domegacmbdThot * np.array(high_desi_planck_1507mK_Terrs) * 1e-4, label="Planck + DESI", linestyle='solid', color='green')
plt.loglog(high_masses, domegacmbdThot * np.array(high_euclid_planck_1507mK_Terrs) * 1e-4 , label="Planck + EUCLID", linestyle='solid', color='blue')

#sigmaline5 = high_masses * (1./93.) * 3. * np.power(1.5072, 2.) * np.power(1.95, -3) * (1.5072/5)
sigmaline5 = omegascmb_hot / 5.
plt.loglog(high_masses, sigmaline5, label=r"$5\sigma$", linestyle='solid', color='black')
np.savetxt(plotdatadir+'1507mK_omega_5sigma.csv', [high_masses, sigmaline5])

plt.title(r"$T_{\chi, 0}$ = 1.5 K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{\omega_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='upper left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
#plt.ylim((-0.001,0.005))
plt.savefig("/Users/nicholasdeporzio/Desktop/plot1.png")
plt.show()

#####################
plt.figure(figsize=(15,7.5))
#plt.loglog(high_masses, np.array(high_boss_cmbs4_905mK_omegaerrs) , label="CMBS4 + BOSS", linestyle='dashed', color='red')
#plt.loglog(high_masses, np.array(high_desi_cmbs4_905mK_omegaerrs), label="CMBS4 + DESI", linestyle='dashed', color='green')
#plt.loglog(high_masses, np.array(high_euclid_cmbs4_905mK_omegaerrs) , label="CMBS4 + EUCLID", linestyle='dashed', color='blue')
#plt.loglog(high_masses, np.array(high_boss_planck_905mK_omegaerrs) , label="Planck + BOSS", linestyle='solid', color='red')
#plt.loglog(high_masses, np.array(high_desi_planck_905mK_omegaerrs), label="Planck + DESI", linestyle='solid', color='green')
#plt.loglog(high_masses, np.array(high_euclid_planck_905mK_omegaerrs) , label="Planck + EUCLID", linestyle='solid', color='blue')

#plt.loglog(low_masses, np.array(low_boss_cmbs4_905mK_omegaerrs) , linestyle='dashed', color='red')
#plt.loglog(low_masses, np.array(low_desi_cmbs4_905mK_omegaerrs), linestyle='dashed', color='green')
#plt.loglog(low_masses, np.array(low_euclid_cmbs4_905mK_omegaerrs) , linestyle='dashed', color='blue')
#plt.loglog(low_masses, np.array(low_boss_planck_905mK_omegaerrs) , linestyle='solid', color='red')
#plt.loglog(low_masses, np.array(low_desi_planck_905mK_omegaerrs), linestyle='solid', color='green')
#plt.loglog(low_masses, np.array(low_euclid_planck_905mK_omegaerrs) , linestyle='solid', color='blue')





plt.loglog(high_masses, domegacmbdTcold * np.array(high_boss_cmbs4_905mK_Terrs) * 1e-4 , label="CMBS4 + BOSS", linestyle='dashed', color='red')
plt.loglog(high_masses, domegacmbdTcold * np.array(high_desi_cmbs4_905mK_Terrs) * 1e-4, label="CMBS4 + DESI", linestyle='dashed', color='green')
plt.loglog(high_masses, domegacmbdTcold * np.array(high_euclid_cmbs4_905mK_Terrs) * 1e-4 , label="CMBS4 + EUCLID", linestyle='dashed', color='blue')
plt.loglog(high_masses, domegacmbdTcold * np.array(high_boss_planck_905mK_Terrs) * 1e-4 , label="Planck + BOSS", linestyle='solid', color='red')
plt.loglog(high_masses, domegacmbdTcold * np.array(high_desi_planck_905mK_Terrs) * 1e-4, label="Planck + DESI", linestyle='solid', color='green')
plt.loglog(high_masses, domegacmbdTcold * np.array(high_euclid_planck_905mK_Terrs) * 1e-4 , label="Planck + EUCLID", linestyle='solid', color='blue')

print("Conversion Factor: ")
print(domegacmbdTcold)
print("T errors: ")
print(np.array(high_euclid_planck_905mK_Terrs))
print("Conversion factor * T errors: ")
print(domegacmbdTcold * np.array(high_euclid_planck_905mK_Terrs))

#sigmaline6 = high_masses * (1./93.) * 3. * np.power(0.9052, 2.) * np.power(1.95, -3) * (1.5072/5)
sigmaline6 = omegascmb_cold / 5.

print("Omega_rec (1sigma line): ")
print(sigmaline6)
print("Reference value of sigma omegas from Fisher: ")
print(np.array(low_euclid_planck_905mK_omegaerrs))
plt.loglog(high_masses, sigmaline6, label=r"$5\sigma$", linestyle='solid', color='black')
np.savetxt(plotdatadir+'905mK_omega_5sigma.csv', [high_masses, sigmaline6])

plt.title(r"$T_{\chi, 0}$ = 0.9 K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{\omega_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='upper left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
#plt.ylim(top=0.007)
plt.savefig("/Users/nicholasdeporzio/Desktop/plot2.png")
plt.show()

#####################
plt.figure(figsize=(15,7.5))

plt.loglog(high_masses, (1/omegascmb_hot) * domegacmbdThot * np.array(high_boss_cmbs4_1507mK_Terrs) * 1e-4 , label="CMBS4 + BOSS", linestyle='dashed', color='red')
plt.loglog(high_masses, (1/omegascmb_hot) * domegacmbdThot * np.array(high_desi_cmbs4_1507mK_Terrs) * 1e-4, label="CMBS4 + DESI", linestyle='dashed', color='green')
plt.loglog(high_masses, (1/omegascmb_hot) * domegacmbdThot * np.array(high_euclid_cmbs4_1507mK_Terrs) * 1e-4 , label="CMBS4 + EUCLID", linestyle='dashed', color='blue')
plt.loglog(high_masses, (1/omegascmb_hot) * domegacmbdThot * np.array(high_boss_planck_1507mK_Terrs) * 1e-4 , label="Planck + BOSS", linestyle='solid', color='red')
plt.loglog(high_masses, (1/omegascmb_hot) * domegacmbdThot * np.array(high_desi_planck_1507mK_Terrs) * 1e-4, label="Planck + DESI", linestyle='solid', color='green')
plt.loglog(high_masses, (1/omegascmb_hot) * domegacmbdThot * np.array(high_euclid_planck_1507mK_Terrs) * 1e-4 , label="Planck + EUCLID", linestyle='solid', color='blue')

sigmaline7 = len(high_masses) * [1./5.]

plt.loglog(high_masses, sigmaline7, label=r"$5\sigma$", linestyle='solid', color='black')

plt.title(r"$T_{\chi, 0}$ = 1.5 K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{\omega_\chi}/\omega_\chi$", fontsize=24)
plt.legend(fontsize=18, loc='upper left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/plot3.png")
plt.show()

#####################
plt.figure(figsize=(15,7.5))

plt.loglog(high_masses, (1/omegascmb_cold) * domegacmbdTcold * np.array(high_boss_cmbs4_905mK_Terrs) * 1e-4 , label="CMBS4 + BOSS", linestyle='dashed', color='red')
plt.loglog(high_masses, (1/omegascmb_cold) * domegacmbdTcold * np.array(high_desi_cmbs4_905mK_Terrs) * 1e-4, label="CMBS4 + DESI", linestyle='dashed', color='green')
plt.loglog(high_masses, (1/omegascmb_cold) * domegacmbdTcold * np.array(high_euclid_cmbs4_905mK_Terrs) * 1e-4 , label="CMBS4 + EUCLID", linestyle='dashed', color='blue')
plt.loglog(high_masses, (1/omegascmb_cold) * domegacmbdTcold * np.array(high_boss_planck_905mK_Terrs) * 1e-4 , label="Planck + BOSS", linestyle='solid', color='red')
plt.loglog(high_masses, (1/omegascmb_cold) * domegacmbdTcold * np.array(high_desi_planck_905mK_Terrs) * 1e-4, label="Planck + DESI", linestyle='solid', color='green')
plt.loglog(high_masses, (1/omegascmb_cold) * domegacmbdTcold * np.array(high_euclid_planck_905mK_Terrs) * 1e-4 , label="Planck + EUCLID", linestyle='solid', color='blue')

sigmaline8 = len(high_masses) * [1./5.]

plt.loglog(high_masses, sigmaline8, label=r"$5\sigma$", linestyle='solid', color='black')

plt.title(r"$T_{\chi, 0}$ = 0.9 K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{\omega_\chi}/\omega_\chi$", fontsize=24)
plt.legend(fontsize=18, loc='upper left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
#plt.ylim(top=0.007)
plt.savefig("/Users/nicholasdeporzio/Desktop/plot4.png")
plt.show()

#####################
plt.figure(figsize=(15,7.5))

plt.loglog(high_masses, (1/omegascmb_hot) * domegacmbdThot * np.array(high_desi_cmbs4_1507mK_Terrs) * 1e-4 , label="CMBS4 + DESI, 1.5K", linestyle='dashed', color='red')
plt.loglog(high_masses, (1/omegascmb_cold) * domegacmbdTcold * np.array(high_desi_cmbs4_905mK_Terrs) * 1e-4, label="CMBS4 + DESI, 0.9K", linestyle='dashed', color='green')


sigmaline9 = len(high_masses) * [1./3.]

plt.loglog(high_masses, sigmaline9, label=r"$3\sigma$", linestyle='solid', color='black')

#plt.title(r"$T_{\chi, 0}$ = 0.9 K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{\omega_\chi}/\omega_\chi$", fontsize=24)
plt.legend(fontsize=18, loc='upper left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/plot5.png")
plt.show()

In [ ]:
#butt
from scipy.integrate import quad
def integrand(p, m, T):
    return (2./np.power(2 * np.pi, 3) * (1/ (np.exp(p/T) + 1)) 
            * np.sqrt(np.power(p, 2) + np.power(m, 2))  * (4 * np.pi * np.power(p, 2)))

omegashot = np.array([quad(integrand, 0, 10, args=(mval, 1.5e-4 ))[0] for mval in high_masses])
omegascold = np.array([quad(integrand, 0, 10, args=(mval, 0.9e-4 ))[0] for mval in high_masses])

omegascmb_hot = np.array([quad(integrand, 0, 10, args=(mval, 1.5*1100*1e-4 ))[0] for mval in high_masses])
omegascmb_cold = np.array([quad(integrand, 0, 10, args=(mval, 0.9*1100*1e-4 ))[0] for mval in high_masses])

relomegaerrhot = omegascmb_hot/omegashot
relomegaerrcold = omegascmb_cold/omegascold

dstepuphot = np.array([quad(integrand, 0, 10, args=(mval, 1.01*1.5*1e-4 ))[0] for mval in high_masses])
dstepdownhot = np.array([quad(integrand, 0, 10, args=(mval, 0.99*1.5*1e-4 ))[0] for mval in high_masses])
dstepupcold = np.array([quad(integrand, 0, 10, args=(mval, 1.01*0.9*1e-4 ))[0] for mval in high_masses])
dstepdowncold = np.array([quad(integrand, 0, 10, args=(mval, 0.99*0.9*1e-4 ))[0] for mval in high_masses])

dstepupcmbhot = np.array([quad(integrand, 0, 10, args=(mval, 1.001*1.5*1100*1e-4 ))[0] for mval in high_masses])
dstepdowncmbhot = np.array([quad(integrand, 0, 10, args=(mval, 0.999*1.5*1100*1e-4 ))[0] for mval in high_masses])
dstepupcmbcold = np.array([quad(integrand, 0, 10, args=(mval, 1.001*0.9*1100*1e-4 ))[0] for mval in high_masses])
dstepdowncmbcold = np.array([quad(integrand, 0, 10, args=(mval, 0.999*0.9*1100*1e-4 ))[0] for mval in high_masses])

domegadThot = (dstepuphot-dstepdownhot)/(2*0.01*1.5*1e-4)
domegadTcold = (dstepupcold-dstepdowncold)/(2*0.01*0.9*1e-4)

domegacmbdThot = (dstepupcmbhot-dstepdowncmbhot)/(2*0.001*1.5*1e-4)
domegacmbdTcold = (dstepupcmbcold-dstepdowncmbcold)/(2*0.001*0.9*1e-4)

dlogomegadlogT_hot = (1.5*1e-4) * domegadThot / omegashot 
dlogomegadlogT_cold = (0.9*1e-4) * domegadTcold / omegascold

dlogomegacmbdlogT_hot = (1.5*1e-4) * domegacmbdThot / omegascmb_hot 
dlogomegacmbdlogT_cold = (0.9*1e-4) * domegacmbdTcold / omegascmb_cold

deltaTdomegacmbdT_hot = domegacmbdThot * high_desi_cmbs4_1507mK_Terrs
deltaTdomegacmbdT_cold = domegacmbdTcold * high_desi_cmbs4_905mK_Terrs

# wriggle
fishmattype='/inv_fullfisher.mat'
omegacdm_high_desi_cmbs4_1507mK = [np.sqrt(pd.read_csv(basepath+'ps4/High_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+fishmattype, sep='\t', header=0).iloc[1,1]) for idx in range(27)]
omegacdm_high_desi_cmbs4_905mK = [np.sqrt(pd.read_csv(basepath+'ps5/High_Masses/Fixed_Kmax/CMBS4/M'+str(idx+1)+fishmattype, sep='\t', header=0).iloc[1,1]) for idx in range(27)]
omegancdm_today_desi_cmbs4_1507mK = domegadThot * high_desi_cmbs4_1507mK_Terrs
omegancdm_today_desi_cmbs4_905mK = domegadTcold * high_desi_cmbs4_905mK_Terrs
#deltaTdomegacmbdT_hot
#deltaTdomegacmbdT_cold


masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))
omegancdm_set1507mK = ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(1.5072*2.726 / 1.95, 3.))
omegacdm_set1507mK = 0.1127 - omegancdm_set1507mK
omegancdm_set905mK = ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(0.9052*2.726 / 1.95, 3.))
omegacdm_set905mK = 0.1127 - omegancdm_set905mK
#omegascmb_hot = np.array([quad(integrand, 0, 10, args=(mval, 1.5*1100*1e-4 ))[0] for mval in high_masses])
#omegascmb_cold = np.array([quad(integrand, 0, 10, args=(mval, 0.9*1100*1e-4 ))[0] for mval in high_masses])

#Data
data1_omegacdm1507 = omegacdm_high_desi_cmbs4_1507mK/omegacdm_set1507mK
data2_omegancdm1507 = omegancdm_today_desi_cmbs4_1507mK/omegancdm_set1507mK
data3_omegancdm1507_cmb  = deltaTdomegacmbdT_hot/omegascmb_hot


plt.figure(figsize=(15,7.5))

plt.loglog(high_masses, data1_omegacdm1507 , label="CMBS4 + DESI, 1.5K, $\omega_{CDM}$", linestyle='solid', color='red')
plt.loglog(high_masses, data2_omegancdm1507, label="CMBS4 + DESI, 1.5K, $\omega_{NCDM, z=0}$", linestyle='solid', color='green')
plt.loglog(high_masses, data3_omegancdm1507_cmb, label="CMBS4 + DESI, 1.5K, $\omega_{NCDM, z=1100}$", linestyle='solid', color='blue')

plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_\omega / \omega$", fontsize=24)
plt.legend(fontsize=18, loc='upper left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/plot6.png")
plt.show()




In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync10/")
classpath = os.path.join(projectdir, "class")

# Combine plots
basepath = '/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps5/High_Masses/Fixed_Kmax/CMBS4/'

mattype='/inv_lssfisher.mat'
desi_1507mK_fishers = [(pd.read_csv(basepath+'M'+str(idx+1)+mattype, sep='\t', header=0).iloc[6,6]) for idx, val in enumerate(np.arange(1, 28))]

masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))
omegas = (masses/93) * np.power(0.9/1.95, 3)
dtdomegas = (93./masses) * np.power(1.95 * 1100, 3) * np.power(0.9 * 1100, -2) * (1./3) 
dtdomegas2 = np.power(dtdomegas, 2.)
relerrs = desi_1507mK_fishers * dtdomegas2 * np.power(omegas, 2)
errs = np.sqrt(np.array(desi_1507mK_fishers))

plt.figure(figsize=(15,7.5))
plt.loglog(masses, errs, linestyle='solid', color='red')
#plt.title("DESI + CMBS4, 0.9[K]")
plt.xlabel("Relic Mass [eV]")
#plt.ylabel(r"$F_{\omega_\chi, \omega_\chi} \times \omega_\chi^2$")
plt.ylabel(r"$\sigma  T$")
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/Plot1.png")
plt.show()

In [ ]:
plt.figure(figsize=(15,7.5))
#plt.semilogx(masses, boss_1500mK_Nerrs, label="BOSS, 1.5072K", linestyle='dotted', color='orange')
#plt.semilogx(masses, boss_1100mK_Nerrs, label="BOSS, 0.9052K", linestyle='dotted', color='green')
plt.semilogx(masses, desi_1500mK_Nerrs, label="DESI, 1.5072K", linestyle='dashed', color='orange')
plt.semilogx(masses, desi_1100mK_Nerrs, label="DESI, 0.9052K", linestyle='dashed', color='green')
#plt.semilogx(masses, euclid_1500mK_Nerrs, label="EUCLID, 1.5072K", color='orange')
#plt.semilogx(masses, euclid_1100mK_Nerrs, label="EUCLID, 0.9052K", color='green')
plt.title(r"$N_{eff}$ Shift, LSS + CMBS4")
plt.xlabel("Relic Mass [eV]")
plt.ylabel(r"$\Delta N_{eff}$")
plt.legend()
plt.tick_params(axis='x', which='minor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/relicforecast_N.png")
plt.show() 

In [ ]:
print(boss_1500mK_Nerrs[0])
print(boss_1100mK_Nerrs[0])
print(desi_1500mK_Nerrs[0])
print(desi_1100mK_Nerrs[0])
print(euclid_1500mK_Nerrs[0])
print(euclid_1100mK_Nerrs[0])

In [ ]:
errors_Tncdm_cmb = [np.sqrt(fval.numpy_cmb_covariance[6,6]) for fidx, fval in enumerate(forecastset)]    
errors_Tncdm_lss = [np.sqrt(fval.numpy_lss_covariance[6,6]) for fidx, fval in enumerate(forecastset)]
errors_Tncdm_full = [np.sqrt(fval.numpy_full_covariance[6,6]) for fidx, fval in enumerate(forecastset)]

# Plot Set 11 - Demonstrate Shift Formalism in b0
# GOOD

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps11_resultsdir = os.path.join(projectdir, 'results', 'ps11')
ps11_convergencedir = os.path.join(ps11_resultsdir, 'convergence')
cf.makedirectory(ps11_resultsdir)
cf.makedirectory(ps11_convergencedir) 

# Set fiducial cosmology
ps11_fid = { # Fiducial for DESI/EUCLID MCMCs
        "A_s" : 2.22e-9, 
        "n_s" : 0.965,
        "omega_b" : 0.02222,
        "omega_cdm" : 0.1120,
        "tau_reio" : 0.06,
        "h" : 0.70,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 3., 
        "T_ncdm" : 1.95/2.726, 
        "m_ncdm" : 0.03, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_vary" : None, #Not used for neutrino forecasts
        } 

# DESI survey parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Run Forecast
ps11_forecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    ps11_fid, 
    z_table, 
    "DESI"
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)
ps11_forecast.gen_pm()
ps11_forecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                          
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2',
        'b_Amp'
    ],
    mu_step=mu_integral_step, 
    skipgen=False)

# Restructure results
DD = np.array(ps11_forecast.fisher[10, 10])
Dj = np.array(ps11_forecast.fisher[0:10, 10])
Extended_Fisher = np.array(ps11_forecast.fisher)
ps11_forecast.fisher = np.array(ps11_forecast.fisher[0:10, 0:10])
ps11_forecast.fisher_order.remove('b_Amp')

# Add CMB Data
ps11_forecast.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'M_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"))

# Save results
ps11_forecast.export_matrices(ps11_resultsdir)
pd.DataFrame(ps11_forecast.numpy_full_fisher, columns=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                          
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ]).to_csv(os.path.join(ps11_resultsdir, "yescmbfisher.csv"), sep='\t')
pd.DataFrame(ps11_forecast.numpy_lss_fisher, columns=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                          
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ]).to_csv(os.path.join(ps11_resultsdir, "nocmbfisher.csv"), sep='\t')

# Print the uncertainty in the P_g shift amplitude
D_spread = np.sqrt(DD)
np.savetxt(path+"shift_root_DD.csv", np.array([D_spread]), delimiter='\t')
print(r"Uncertainty in P_g shift amplitude: ", D_spread)

#Djvec1=(ps11_forecast.numpy_full_fisher[8,:]  *  0.001)

# Print parameter errors
for i in range(10): 
    print(np.sqrt(ps11_forecast.numpy_full_covariance[i, i]), '\n')

# Print shift amplitude
#Amp_Projection[4] = 0.
pd.DataFrame(np.array(Dj)).to_csv(os.path.join(ps11_resultsdir, "shift_basis_projection.csv"), sep='\t')
print("Amplitude vector, <D, g_i>:")
print(pd.DataFrame(np.array(Dj)))

# Print parameter shift  
Delta_A = np.matmul(np.linalg.inv(np.array(ps11_forecast.numpy_full_fisher)), Dj)
print('Parameter shift vector, delta a_i: ')
pd.DataFrame(np.array(Delta_A)).to_csv(os.path.join(ps11_resultsdir, "shift_parameter_amplitude.csv"), sep='\t')
print(pd.DataFrame(np.array(Delta_A)))
print("Parameter shift/error vector, (delta a_i / sigma_i): ")
params = [
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ]
for i in range(10): 
    print("delta a_i / sigma_i for", params[i], ": ", (Delta_A[i]
                                                       /np.sqrt(np.linalg.inv(np.array(ps11_forecast.numpy_full_fisher))[i,i])))
dill.dump_session(os.path.join(ps11_resultsdir, 'ps11.db'))

# Plot Set 12 - Forecast Shift in Neutrino Cosmology w/ and w/o RelicFast
# GOOD



In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync4")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps12_resultsdir = os.path.join(projectdir, 'results', 'ps12')
ps12_convergencedir = os.path.join(ps12_resultsdir, 'convergence')
cf.makedirectory(ps12_resultsdir)
cf.makedirectory(ps12_convergencedir) 

# Set fiducial cosmology
#ps12_fid = { # Fiducial for DESI/EUCLID MCMCs
#       "A_s" : 2.22e-9, 
#        "n_s" : 0.965,
#        "omega_b" : 0.02222,
#        "omega_cdm" : 0.1120,
#        "tau_reio" : 0.06,
#        "h" : 0.70,
#        "T_cmb" : 2.726, # Units [K]
#        "N_ncdm" : 3., 
#        "T_ncdm" : 1.95/2.726, 
#        "m_ncdm" : 0.03, # Units [eV]
#        "b0" : 1.0, 
#        "alphak2" : 1.0,
#        "sigma_fog_0" : 250000, #Units [m s^-2]
#        "N_eff" : 3.046 - (3 * 1.0132), 
#        "relic_vary" : None, #Not used for neutrino forecasts
#        } 

# Linda Fiducial Cosmology 
ps12_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (0.9052/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "T_ncdm" # Fix T_ncdm or m_ncdm 
        }

# DESI survey parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# EUCLID values
#z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
#dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, 
#    2308.975, 1514.831, 1474.707, 893.716, 497.613])
#skycover = 0.3636 # Sky coverage of survey in fraction
# Run Fisher Forecast

masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps12_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps12_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps12_fiducialset = [dict(ps12_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps12_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps12_fiducialset)]
for fidx, fcst in enumerate(ps12_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm',                                 
            'sigma_fog',                                   
            'b0',                                         
            'alpha_k2', 
            'D_Amp'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")
    # Save notebook state
    dill.dump_session(os.path.join(ps12_resultsdir, 'ps12.db'))
    
import datetime
print(str(datetime.datetime.now()))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps12/ps12.db')

# Save results 
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherCMBS4_bin"
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/New_Relic_CMB_Fisher_Matrices/FisherPlanck_bin"
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t b0 \t alpha_k2 \t D_Amp'
for fidx, fval in enumerate(ps12_forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm'],
#        fisherpath=inpath+str(fidx+1)+"_T0.9K.txt",
        fisherpath=inpath+str(1)+"_T0.9K.txt")
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    outdir=os.path.join(ps12_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + '_Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)

In [ ]:
# Restructure results
DD = np.array(ps12_forecast.fisher[10, 10])
Dj = np.array(ps12_forecast.fisher[0:10, 10])
Extended_Fisher = np.array(ps12_forecast.fisher)
ps12_forecast.fisher = np.array(ps12_forecast.fisher[0:10, 0:10])
ps12_forecast.fisher_order.remove('D_Amp')

# Save notebook state
dill.dump_session(os.path.join(ps12_resultsdir, 'ps12.db'))

# Add CMB Data
ps12_forecast.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'M_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"))

# Save results
ps12_forecast.export_matrices(ps12_resultsdir)
pd.DataFrame(ps12_forecast.numpy_full_fisher, columns=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                    
        'alpha_k2'
    ]).to_csv(os.path.join(ps12_resultsdir, "yescmbfisher.csv"), sep='\t')
pd.DataFrame(ps12_forecast.numpy_lss_fisher, columns=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                     
        'alpha_k2'
    ]).to_csv(os.path.join(ps12_resultsdir, "nocmbfisher.csv"), sep='\t')

# Print the uncertainty in the P_g shift amplitude
D_spread = np.sqrt(DD)
np.savetxt(os.path.join(ps12_resultsdir, "shift_root_DD.csv"), np.array([D_spread]), delimiter='\t')
print(r"Uncertainty in P_g shift amplitude: ", D_spread)

#Djvec1=(ps12_forecast.numpy_full_fisher[8,:]  *  0.001)

# Print parameter errors
for i in range(10): 
    print(np.sqrt(ps12_forecast.numpy_full_covariance[i, i]), '\n')

# Print shift amplitude
#Amp_Projection[4] = 0.
pd.DataFrame(np.array(Dj)).to_csv(os.path.join(ps12_resultsdir, "shift_basis_projection.csv"), sep='\t')
print("Amplitude vector, <D, g_i>:")
print(pd.DataFrame(np.array(Dj)))

# Print parameter shift  
Delta_A = np.matmul(np.linalg.inv(np.array(ps12_forecast.numpy_full_fisher)), Dj)
print('Parameter shift vector, delta a_i: ')
pd.DataFrame(np.array(Delta_A)).to_csv(os.path.join(ps12_resultsdir, "shift_parameter_amplitude.csv"), sep='\t')
print(pd.DataFrame(np.array(Delta_A)))
print("Parameter shift/error vector, (delta a_i / sigma_i): ")

In [ ]:
params = [
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',
        'alpha_k2'
    ]
for i in range(10): 
    print("delta a_i / sigma_i for", params[i], ": ", (Delta_A[i]
                                                       /np.sqrt(np.linalg.inv(np.array(ps12_forecast.numpy_full_fisher))[i,i])))



# Plot Set 13 - Forecast g_ncdm for DESI at 1.5072 [K]

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Volumes/SSD01/data.nosync13/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps13_resultsdir = os.path.join(projectdir, 'results', 'ps13_test')
#ps13_convergencedir = os.path.join(ps13_resultsdir, 'convergence')
cf.makedirectory(ps13_resultsdir)
#cf.makedirectory(ps13_convergencedir) 

# Linda Fiducial Cosmology 
ps13_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "deg_ncdm" : 1,
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# DESI Parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Run Fisher Forecast
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps13_fid['omega_cdm'] - ps13_fid["N_ncdm"]*((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps13_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps13_fiducialset = [dict(ps13_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps13_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps13_fiducialset)]
for fidx, fcst in enumerate(ps13_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'N_ncdm',                                 
            'sigma_fog',                                   
            'b0',                                         
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")
    dill.dump_session(os.path.join(ps13_resultsdir, 'ps13.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps13/ps13.db')

In [ ]:
prepath = "/Users/nicholasdeporzio/Desktop/Fisher_CMB/Fisher_T1.5K/FisherCMBS4_bin"
postpath = "_T1.5K.txt"

for fidx, fval in enumerate(ps13_forecastset):
    fval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'N_ncdm', 'sigma_fog', 'b0', 'alpha_k2'])
    fval.load_cmb_fisher(
        fisher_order=[
            'A_s',                                    
            'h',                                  
            'n_s',                                        
            'omega_b',                                        
            'omega_cdm',                                   
            'tau_reio',                                                                             
            'N_ncdm'],
        fisherpath = prepath + str(fidx+1) + postpath
    )
    outdir=os.path.join(ps13_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

masses_ev = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

planckpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps13/Planck/M"
cmbs4path = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps13/CMBS4/M"
postpath = "/inv_lssfisher.mat"

data1 = [np.sqrt(np.loadtxt(planckpath+str(idx+1)+postpath, delimiter='\t', skiprows=1)[6,6]) for idx in range(27)]
data2 = [np.sqrt(np.loadtxt(cmbs4path+str(idx+1)+postpath, delimiter='\t', skiprows=1)[6,6]) for idx in range(27)]

plt.figure(figsize=(15,7.5))

plt.semilogx(masses_ev, data1, label=r"$\sigma_{g_\chi}$, DESI+Planck 1.5K", linestyle='solid', color='red')
plt.semilogx(masses_ev, data2, label=r"$\sigma_{g_\chi}$, DESI+CMBS4, 1.5K", linestyle='solid', color='green')

plt.title("T=1.5K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/testing/plotA5.png")
plt.show()

# Plot Set 14 - Forecast g_ncdm for DESI at 0.9052 [K]

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync14/")
classpath = os.path.join(projectdir, "class2")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps14_resultsdir = os.path.join(projectdir, 'results', 'ps14')
cf.makedirectory(ps14_resultsdir)

# Linda Fiducial Cosmology 
ps14_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "deg_ncdm" : 1,
        "T_ncdm" : (0.9052/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# DESI Parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Run Fisher Forecast
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps14_fid['omega_cdm'] - ps14_fid["N_ncdm"]*((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps14_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps14_fiducialset = [dict(ps14_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps14_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps14_fiducialset)]
for fidx, fcst in enumerate(ps14_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'N_ncdm',                                 
            'sigma_fog',                                   
            'b0',                                         
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")
    dill.dump_session(os.path.join(ps14_resultsdir, 'ps14.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps14/ps14.db')

In [ ]:
prepath = "/Users/nicholasdeporzio/Desktop/Fisher_CMB/Fisher_T0.9K/FisherPlanck_bin"
postpath = "_T0.9K.txt"

for fidx, fval in enumerate(ps14_forecastset):
    fval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'N_ncdm', 'sigma_fog', 'b0', 'alpha_k2'])
    fval.load_cmb_fisher(
        fisher_order=[
            'A_s',                                    
            'h',                                  
            'n_s',                                        
            'omega_b',                                        
            'omega_cdm',                                   
            'tau_reio',                                                                             
            'N_ncdm'],
        fisherpath = prepath + str(fidx+1) + postpath
    )
    outdir=os.path.join(ps14_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)

In [ ]:
masses_ev = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

planckpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps14/Planck/M"
cmbs4path = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps14/CMBS4/M"
postpath = "/inv_fullfisher.mat"

data1 = [np.sqrt(np.loadtxt(planckpath+str(idx+1)+postpath, delimiter='\t', skiprows=1)[6,6]) for idx in range(27)]
data2 = [np.sqrt(np.loadtxt(cmbs4path+str(idx+1)+postpath, delimiter='\t', skiprows=1)[6,6]) for idx in range(27)]

plt.figure(figsize=(15,7.5))

plt.semilogx(masses_ev, data1, label=r"$\sigma_{g_\chi}$, DESI+Planck 0.9K", linestyle='solid', color='red')
plt.semilogx(masses_ev, data2, label=r"$\sigma_{g_\chi}$, DESI+CMBS4, 0.9K", linestyle='solid', color='green')

plt.title("T=0.9K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/testing/plotA6.png")
plt.show()

# Plot Set 15 - Forecast g_ncdm for EUCLID at 1.5072 [K]

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/data.nosync15/")
classpath = os.path.join(projectdir, "class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps15_resultsdir = os.path.join(projectdir, 'results', 'ps15')
#ps15_convergencedir = os.path.join(ps15_resultsdir, 'convergence')
cf.makedirectory(ps15_resultsdir)
#cf.makedirectory(ps15_convergencedir) 

# Linda Fiducial Cosmology 
ps15_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "deg_ncdm" : 1,
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "beta0" : 1.7, 
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# EUCLID Parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, 
    2308.975, 1514.831, 1474.707, 893.716, 497.613])
skycover = 0.3636 # Sky coverage of survey in fraction

# Run Fisher Forecast
masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

omegacdm_set = ps15_fid['omega_cdm'] - ps15_fid["N_ncdm"]*((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps15_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps15_fiducialset = [dict(ps15_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps15_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "EUCLID",
    dNdz, 
    fsky=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps15_fiducialset)]
for fidx, fcst in enumerate(ps15_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'N_ncdm',                                 
            'sigma_fog',                                   
            'beta0',
            'beta1',
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")
    dill.dump_session(os.path.join(ps15_resultsdir, 'ps15.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps15/ps15.db')

In [ ]:
prepath = "/Users/nicholasdeporzio/Desktop/Fisher_CMB/Fisher_T1.5K/FisherCMBS4_bin"
postpath = "_T1.5K.txt"
head = 'A_s \t h \t n_s \t omega_b \t omega_cdm \t tau_reio \t N_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'

for fidx, fval in enumerate(ps15_forecastset):
    fval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'N_ncdm', 'sigma_fog', 'beta0', 'beta1', 'alpha_k2'])
    fval.load_cmb_fisher(
        fisher_order=[
            'A_s',                                    
            'h',                                  
            'n_s',                                        
            'omega_b',                                        
            'omega_cdm',                                   
            'tau_reio',                                                                             
            'N_ncdm'],
        fisherpath = prepath + str(fidx+1) + postpath
    )
    outdir=os.path.join(ps15_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + '_Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)
    fval.pandas_lss_fisher.to_csv(outdir + "_LSS_Fisher.dat", sep='\t', index=False)



In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

masses_ev = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

planckpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps15/Planck/M"
cmbs4path = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps15/CMBS4/M"
postpath = "/inv_fullfisher.mat"

data1 = [np.sqrt(np.loadtxt(planckpath+str(idx+1)+postpath, delimiter='\t', skiprows=1)[6,6]) for idx in range(27)]
data2 = [np.sqrt(np.loadtxt(cmbs4path+str(idx+1)+postpath, delimiter='\t', skiprows=1)[6,6]) for idx in range(27)]

plt.figure(figsize=(15,7.5))

plt.semilogx(masses_ev, data1, label=r"$\sigma_{g_\chi}$, EUCLID+Planck 1.5K", linestyle='solid', color='red')
plt.semilogx(masses_ev, data2, label=r"$\sigma_{g_\chi}$, EUCLID+CMBS4, 1.5K", linestyle='solid', color='green')

plt.title("T=1.5K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/testing/plotA7.png")
plt.show()

# Plot Set 16 - Forecast g_ncdm for EUCLID at 0.9052 [K]

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
#projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
projectdir = cf.correct_path("/Volumes/SSD01/")
datastore = cf.correct_path("/Volumes/SSD01/data.nosync16p/")
classpath = os.path.join("/Users/nicholasdeporzio/Desktop/cfworkspace/class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps16_resultsdir = os.path.join(projectdir, 'results', 'ps16p')
cf.makedirectory(ps16_resultsdir)

# Linda Fiducial Cosmology 
ps16_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 0., 
        "deg_ncdm" : 0.,
        "T_ncdm" : (0.9052/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "beta0" : 1.7, 
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# EUCLID Parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, 
    2308.975, 1514.831, 1474.707, 893.716, 497.613])
skycover = 0.3636 # Sky coverage of survey in fraction

# Run Fisher Forecast
#masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))
masses = np.array([0.2])

omegacdm_set = ps16_fid['omega_cdm'] - ps16_fid["N_ncdm"]*((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps16_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps16_fiducialset = [dict(ps16_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
ps16_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "EUCLID",
    dNdz, 
    fsky=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps16_fiducialset)]
for fidx, fcst in enumerate(ps16_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            #'N_ncdm',                                 
            'sigma_fog',                                   
            'beta0',
            'beta1',
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")
    dill.dump_session(os.path.join(ps16_resultsdir, 'ps16p.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps16_test/ps16o.db')

In [ ]:
np.savetxt("/Users/nicholasdeporzio/Desktop/k_0.txt", ps16_forecastset[0].k_table[0], delimiter='\t')
np.savetxt("/Users/nicholasdeporzio/Desktop/db_0.txt", ps16_forecastset[0].spectra_mid[0].b_interp_table, delimiter='\t')
np.savetxt("/Users/nicholasdeporzio/Desktop/dcdm_0.txt", ps16_forecastset[0].spectra_mid[0].cdm_interp_table, delimiter='\t')
np.savetxt("/Users/nicholasdeporzio/Desktop/Pg_0.txt", ps16_forecastset[0].Pg[0, :, 20], delimiter='\t')
np.savetxt("/Users/nicholasdeporzio/Desktop/Pm_0.txt", ps16_forecastset[0].Pm[0], delimiter='\t')



In [ ]:
print(masses[5])
ps16_forecastset[5].spectra_mid[0].datadirectory


In [ ]:
prepath = "/Users/nicholasdeporzio/Desktop/Fisher_CMB/Fisher_T0.9K/FisherCMBS4_bin"
postpath = "_T0.9K.txt"
head = 'A_s \t h \t n_s \t omega_b \t omega_cdm \t tau_reio \t N_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'

for fidx, fval in enumerate(ps16_forecastset):
    fval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'N_ncdm', 'sigma_fog', 'beta0', 'beta1', 'alpha_k2'])
    fval.load_cmb_fisher(
        fisher_order=[
            'A_s',                                    
            'h',                                  
            'n_s',                                        
            'omega_b',                                        
            'omega_cdm',                                   
            'tau_reio',                                                                             
            'N_ncdm'],
        fisherpath = prepath + str(fidx+1) + postpath
    )
    outdir=os.path.join(ps16_resultsdir, 'M'+str(fidx+1))
    cf.makedirectory(outdir)
    fval.export_matrices(outdir)
    np.savetxt(outdir + '_Full_Fisher.dat', fval.numpy_full_fisher, delimiter='\t', header=head)
    fval.pandas_lss_fisher.to_csv(outdir + "_LSS_Fisher.dat", sep='\t', index=False)
    

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

masses_ev = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))

planckpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps16/Planck/M"
cmbs4path = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps16/CMBS4/M"
postpath = "/inv_fullfisher.mat"

data1 = [np.sqrt(np.loadtxt(planckpath+str(idx+1)+postpath, delimiter='\t', skiprows=1)[6,6]) for idx in range(27)]
data2 = [np.sqrt(np.loadtxt(cmbs4path+str(idx+1)+postpath, delimiter='\t', skiprows=1)[6,6]) for idx in range(27)]

plt.figure(figsize=(15,7.5))

plt.semilogx(masses_ev, data1, label=r"$\sigma_{g_\chi}$, EUCLID+Planck 0.9K", linestyle='solid', color='red')
plt.semilogx(masses_ev, data2, label=r"$\sigma_{g_\chi}$, EUCLID+CMBS4, 0.9K", linestyle='solid', color='green')

plt.title("T=0.9K", fontsize=24)
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/testing/plotA8.png")
plt.show()

# Plot Set 17 - Grid Plot for DESI

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
#projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Volumes/SSD01/data.nosync17/")
classpath = os.path.join("/Users/nicholasdeporzio/Desktop/cfworkspace/class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps17_resultsdir = cf.correct_path("/Volumes/SSD01/results/ps17/")


#ps17_convergencedir = os.path.join(ps17_resultsdir, 'convergence')
cf.makedirectory(ps17_resultsdir)
#cf.makedirectory(ps17_convergencedir) 

# Linda Fiducial Cosmology 
ps17_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "deg_ncdm" : 1,
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# DESI Parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Run Fisher Forecast
masses = np.geomspace(0.01, 10., 10)
temps = np.linspace(0.79, 1.5, 12)

omegacdm_set = np.array([ps17_fid['omega_cdm'] - ps17_fid["N_ncdm"]*((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(tval / 1.95, 3.)) for tidx, tval in enumerate(temps)])                                  
ps17_fiducialset = [[dict(ps17_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[tidx, midx], 'T_ncdm' : temps[tidx]/2.726}) 
               for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)] 
ps17_forecastset = [[cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(fidrowvals)] for fidrowidx, fidrowvals in enumerate(ps17_fiducialset)]
for frowidx, frowval in enumerate(ps17_forecastset): 
    for fidx, fcst in enumerate(frowval): 
        fcst.gen_pm()
        fcst.gen_fisher(
            fisher_order=[
                'omega_b',                                    
                'omega_cdm',                                  
                'n_s',                                        
                'A_s',                                        
                'tau_reio',                                   
                'h',                                                                             
                'N_ncdm',                                 
                'sigma_fog',                                   
                'b0',                                         
                'alpha_k2'],
            mu_step=mu_integral_step, 
            skipgen=False)
        print("Relic Forecast ", fidx, " complete...")
        dill.dump_session(os.path.join(ps17_resultsdir, 'Apr10_'+str(frowidx)+'_'+str(fidx)+'_ps17.db'))

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/ClusterResults_2020_04_11/ps17/52270113/11_9_ps17.db')

ps17_resultsdir = cf.correct_path("/Volumes/SSD01/results/ps17/")

cmbsurvey="CMBS4"
#cmbsurvey="Planck"

prepath = cf.priors_directory()
tempnames = ['0.79', '0.85', '0.92', '0.98', '1.05', '1.11', '1.18', '1.24', '1.31', '1.37', '1.44', '1.50']
#head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t N_ncdm \t sigma_fog \t beta0 \t alpha_k2'
head = 'A_s \t h \t n_s \t omega_b \t omega_cdm \t tau_reio \t N_ncdm \t sigma_fog \t beta0 \t alpha_k2'

for tidx, tval in enumerate(ps17_forecastset):
    for midx, mval in enumerate(tval): 
        mval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'N_ncdm', 'sigma_fog', 'b0', 'alpha_k2'])
        mval.load_cmb_fisher(
            fisher_order=[
                'A_s',
                'h',  
                'n_s',
                'omega_b',                                    
                'omega_cdm',                                                     
                'tau_reio',                                                                                    
                'N_ncdm'],
            fisherpath = cf.priors_directory()+'/Relic_Grid_Priors/Fisher_T'+tempnames[tidx]+'K/Fisher'+cmbsurvey+'_bin'+str(midx+1)+'_T'+tempnames[tidx]+'K.txt'
        )
        outdir=os.path.join(ps17_resultsdir, cmbsurvey, 'T'+tempnames[tidx], 'M'+str(midx+1))
        cf.makedirectory(outdir)
        mval.export_matrices(outdir)
        np.savetxt(outdir + '_Full_Fisher.dat', mval.numpy_full_fisher, delimiter='\t', header=head)
        mval.pandas_lss_fisher.to_csv(outdir + "_LSS_Fisher.dat", sep='\t', index=False)

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt

#cmbsurvey="CMBS4"
cmbsurvey="Planck"

#dtype="cmb"
#dtype="lss"
dtype="full"

paramidx = 6

if dtype=='lss': 
    title = 'DESI Only'
    postpend = '_lss.png'
elif dtype=='cmb': 
    title =  cmbsurvey + ' Only'
    postpend = '_cmb.png'
elif dtype=='full': 
    title = 'DESI + ' + cmbsurvey
    postpend = '_full.png'

tempnames = ['0.79', '0.85', '0.92', '0.98', '1.05', '1.11', '1.18', '1.24', '1.31', '1.37', '1.44', '1.50']
masses = np.geomspace(0.01, 10., 10)
temps = np.linspace(0.79, 1.5, 12)

datapath = '/Volumes/SSD01/results/ps17'

data = np.array([[np.sqrt(np.loadtxt(
    datapath+'/'+cmbsurvey+'/T'+tempnames[tidx]+'/M'+str(midx+1)+'/inv_'+dtype+'fisher.mat', delimiter='\t', skiprows=1)[paramidx, paramidx])
                               for midx, mval in enumerate(masses)] 
                               for tidx, tval in enumerate(temps)]) 

flatdata = data.flatten()
flatnsigmas = 1./flatdata
logflatdata = np.log10(flatdata)
logflatnsigmas = np.log10(flatnsigmas)




plt.figure(figsize=(15,7.5))

for tidx, tval in enumerate(temps): 
    plt.semilogx(masses, data[tidx, :], 'o', label=r"$\sigma_{g_\chi}$, DESI, " + str(f"{tval:.2f}"), linestyle='solid')

plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{g_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.title(title, fontsize=22)
plt.savefig(datapath+'/'+cmbsurvey+'/plotA17'+postpend)
plt.show()

plt.figure(figsize=(25,25))    
# Create long format
people = np.repeat(np.flip(temps), len(masses))
feature = list(masses) * len(temps)
df=pd.DataFrame({'Mass [eV]': feature, 'Temperature [K]': people, 'value': flatdata })
# plot it
df_wide=df.pivot_table( index='Temperature [K]', columns='Mass [eV]', values='value' )
ax=sns.heatmap(df_wide, 
               yticklabels=(np.flip(temps)),
               #xticklabels=(np.flip(masses)),
               cbar_kws={'label': r'$\sigma_{g_\chi}$'}, 
               #vmin=0.2, 
               #vmax=1.5,
               annot=True
              )

# format text labels
fmt = '{:0.2f}'
xticklabels = []
for item in ax.get_xticklabels():
    item.set_text(fmt.format(float(item.get_text())))
    xticklabels += [item]
yticklabels = []
for item in ax.get_yticklabels():
    item.set_text(fmt.format(float(item.get_text())))
    yticklabels += [item]

ax.set_xticklabels(xticklabels, fontsize=18)
ax.set_yticklabels(yticklabels, fontsize=18)

ax.set_xlabel(ax.get_xlabel(), fontsize=18)
ax.set_ylabel(ax.get_ylabel(), fontsize=18)

cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=18)
ax.figure.axes[-1].set_ylabel(ax.figure.axes[-1].get_ylabel(), size=22)

plt.title(title, fontsize=22)
plt.savefig(datapath+'/'+cmbsurvey+'/plotA17b'+postpend)
plt.show()

# Plot Set 18 - Minimum Relic Temp Fine Plots for DESI

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Volumes/SSD01/data.nosync18/")
classpath = os.path.join(projectdir, "class2")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 


# Generate output paths  
#ps18_resultsdir = os.path.join(projectdir, 'results', 'ps18')
ps18_resultsdir = cf.correct_path("/Volumes/SSD01/results/ps18/")


#ps18_convergencedir = os.path.join(ps18_resultsdir, 'convergence')
cf.makedirectory(ps18_resultsdir)
#cf.makedirectory(ps18_convergencedir) 

# Linda Fiducial Cosmology 
ps18_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "deg_ncdm" : 1,
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# DESI Parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Run Fisher Forecast
masses = np.geomspace(0.01, 10., 21)
temps = np.array([0.79, 0.91, 0.94, 1.08])

omegacdm_set = np.array([ps18_fid['omega_cdm'] - ps18_fid["N_ncdm"]*((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(tval / 1.95, 3.)) for tidx, tval in enumerate(temps)])                                  
ps18_fiducialset = [[dict(ps18_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[tidx, midx], 'T_ncdm' : temps[tidx]/2.726}) 
               for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)] 
ps18_forecastset = [[cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(fidrowvals)] for fidrowidx, fidrowvals in enumerate(ps18_fiducialset)]



In [ ]:
import dill
#dill.load_session('/Volumes/SSD01/results/ps18/3_3_ps18.db')
for frowidx, frowval in enumerate(ps18_forecastset):
    for fidx, fcst in enumerate(frowval):
        if type(fcst.fisher)==type(None):
            fcst.gen_pm()
            fcst.gen_fisher(
                fisher_order=[
                    'omega_b',
                    'omega_cdm',
                    'n_s',
                    'A_s',
                    'tau_reio',
                    'h',
                    'N_ncdm',
                    'sigma_fog',
                    'b0',
                    'alpha_k2'],
                mu_step=mu_integral_step,
                skipgen=False)
            print("Relic Forecast ", fidx, " complete...")
            dill.dump_session(os.path.join(ps18_resultsdir, str(frowidx)+'_'+str(fidx)+'_ps18.db'))
        else:
            print('Fisher matrix already generated!')

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/cluster/ps18/ps18.db')

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

data = np.array([[np.sqrt(
            np.linalg.inv(
                np.delete(np.delete(fval.fisher, 4, axis=0), 4, axis=1))[5,5])
                               for fidx, fval in enumerate(frowval)] 
                               for frowidx, frowval in enumerate(ps18_forecastset)]) 

flatdata = data.flatten()
flatnsigmas = 1./flatdata
logflatdata = np.log10(flatdata)
logflatnsigmas = np.log10(flatnsigmas)


plt.figure(figsize=(15,7.5))

plt.semilogx(masses, data[0, :], label=r"$\sigma_{g_\chi}$, DESI, 0.79K", linestyle='solid')
plt.semilogx(masses, data[1, :], label=r"$\sigma_{g_\chi}$, DESI, 0.91K", linestyle='solid')
plt.semilogx(masses, data[2, :], label=r"$\sigma_{g_\chi}$, DESI, 0.94K", linestyle='solid')
plt.semilogx(masses, data[3, :], label=r"$\sigma_{g_\chi}$, DESI, 1.08K", linestyle='solid')

plt.xlabel(r"$M_\chi$ [eV ]", fontsize=24)
plt.ylabel(r"$\sigma_{g_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/testing/plotA18a.png")
plt.show()


plt.figure(figsize=(15,7.5))

plt.loglog(masses, 1./data[0, :], label=r"$\sigma_{g_\chi}$, DESI, 0.79K", linestyle='solid')
plt.loglog(masses, 1./data[1, :], label=r"$\sigma_{g_\chi}$, DESI, 0.91K", linestyle='solid')
plt.loglog(masses, 1./data[2, :], label=r"$\sigma_{g_\chi}$, DESI, 0.94K", linestyle='solid')
plt.loglog(masses, 1./data[3, :], label=r"$\sigma_{g_\chi}$, DESI, 1.08K", linestyle='solid')

plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$1/\sigma_{g_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/testing/plotA18b.png")
plt.show()

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/ClusterResults_2020_04_11/ps18/52270117/3_20_ps18.db')

ps18_resultsdir = cf.correct_path("/Volumes/SSD01/results/ps18/")

#cmbsurvey="CMBS4"
cmbsurvey="Planck"

prepath = cf.priors_directory()
tempnames = ['0.79', '0.91', '0.94', '1.08']
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t N_ncdm \t sigma_fog \t beta0 \t alpha_k2'


for tidx, tval in enumerate(ps18_forecastset):
    for midx, mval in enumerate(tval): 
        #mval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'N_ncdm', 'sigma_fog', 'b0', 'alpha_k2'])
        mval.load_cmb_fisher(
            fisher_order=[
                'omega_b',                                    
                'omega_cdm',                                  
                'n_s',                                        
                'A_s',                                        
                'tau_reio',                                   
                'h',                                                                             
                'N_ncdm'],
            fisherpath = cf.priors_directory()+'/Relic_Fine_Priors/Fisher_T'+tempnames[tidx]+'K/Fisher'+cmbsurvey+'_bin'+str(midx+1)+'_T'+tempnames[tidx]+'K.txt'
        )
        outdir=os.path.join(ps18_resultsdir, cmbsurvey, 'T'+tempnames[tidx], 'M'+str(midx+1))
        cf.makedirectory(outdir)
        mval.export_matrices(outdir)
        np.savetxt(outdir + '_Full_Fisher.dat', mval.numpy_full_fisher, delimiter='\t', header=head)
        mval.pandas_lss_fisher.to_csv(outdir + "_LSS_Fisher.dat", sep='\t', index=False)

# Plot Set 19 - Grid Plot for EUCLID

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import cosmicfish as cf
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
#projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Volumes/SSD01/data.nosync19/")
classpath = os.path.join("/Users/nicholasdeporzio/Desktop/cfworkspace/class3")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
#ps19_resultsdir = os.path.join(projectdir, 'results', 'ps19')
ps19_resultsdir = cf.correct_path("/Volumes/SSD01/results/ps19/")
#ps19_convergencedir = os.path.join(ps19_resultsdir, 'convergence')
cf.makedirectory(ps19_resultsdir)
#cf.makedirectory(ps19_convergencedir) 

# Linda Fiducial Cosmology 
ps19_fid = {
        "A_s" : 2.2321e-9,
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1.,
        "deg_ncdm" : 1,
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0,
        "beta0" : 1.7,
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# EUCLID values
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309,
    2308.975, 1514.831, 1474.707, 893.716, 497.613])
skycover = 0.3636 # Sky coverage of survey in fraction

# Run Fisher Forecast
#masses = np.geomspace(0.01, 10., 10)
#temps = np.linspace(0.79, 1.5, 12)
masses = np.array([0.01])
temps = np.array([0.79])

omegacdm_set = np.array([ps19_fid['omega_cdm'] - ps19_fid["N_ncdm"]*((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(tval / 1.95, 3.)) for tidx, tval in enumerate(temps)])                 
ps19_fiducialset = [[dict(ps19_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[tidx, midx], 'T_ncdm' : temps[tidx]/2.726})
               for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
ps19_forecastset = [[cf.forecast(
    classpath,
    datastore,
    'relic',
    fidval,
    z_table,
    "EUCLID",
    dNdz,
    fsky=skycover,
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(fidrowvals)] for fidrowidx, fidrowvals in enumerate(ps19_fiducialset)]

#dill.load_session('/n/home02/ndeporzio/projects/cosmicfish/cfworkspace/results/51392649/2_9_ps19.db')
for frowidx, frowval in enumerate(ps19_forecastset):
    for fidx, fcst in enumerate(frowval):
        if type(fcst.fisher)==type(None):
            fcst.gen_pm()
            fcst.gen_fisher(
                fisher_order=[
                    'omega_b',
                    'omega_cdm',
                    'n_s',
                    'A_s',
                    'tau_reio',
                    'h',                                                                    
                    'N_ncdm',
                    'sigma_fog',
                    'beta0',
                    'beta1',
                    'alpha_k2'],
                mu_step=mu_integral_step,
                skipgen=False)
            print("Relic Forecast ", fidx, " complete...")
            dill.dump_session(os.path.join(ps19_resultsdir, str(frowidx)+'_'+str(fidx)+'_ps19.db'))
        else:
            print('Fisher matrix already generated!')

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/ClusterResults_2020_04_11/ps19/52270118/11_9_ps19.db')

ps19_resultsdir = cf.correct_path("/Volumes/SSD01/results/ps19/")

cmbsurvey="CMBS4"
#cmbsurvey="Planck"

prepath = cf.priors_directory()
tempnames = ['0.79', '0.85', '0.92', '0.98', '1.05', '1.11', '1.18', '1.24', '1.31', '1.37', '1.44', '1.50']
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t N_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'


for tidx, tval in enumerate(ps19_forecastset):
    for midx, mval in enumerate(tval): 
        #mval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'N_ncdm', 'sigma_fog', 'b0', 'alpha_k2'])
        mval.load_cmb_fisher(
            fisher_order=[
                'omega_b',                                    
                'omega_cdm',                                  
                'n_s',                                        
                'A_s',                                        
                'tau_reio',                                   
                'h',                                                                             
                'N_ncdm'],
            fisherpath = cf.priors_directory()+'/Relic_Grid_Priors/Fisher_T'+tempnames[tidx]+'K/Fisher'+cmbsurvey+'_bin'+str(midx+1)+'_T'+tempnames[tidx]+'K.txt'
        )
        outdir=os.path.join(ps19_resultsdir, cmbsurvey, 'T'+tempnames[tidx], 'M'+str(midx+1))
        cf.makedirectory(outdir)
        mval.export_matrices(outdir)
        np.savetxt(outdir + '_Full_Fisher.dat', mval.numpy_full_fisher, delimiter='\t', header=head)
        mval.pandas_lss_fisher.to_csv(outdir + "_LSS_Fisher.dat", sep='\t', index=False)

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt

#cmbsurvey="CMBS4"
cmbsurvey="Planck"

#dtype="cmb"
#dtype="lss"
dtype="full"

paramidx = 6

if dtype=='lss': 
    title = 'EUCLID Only'
    postpend = '_lss.png'
elif dtype=='cmb': 
    title =  cmbsurvey + ' Only'
    postpend = '_cmb.png'
elif dtype=='full': 
    title = 'EUCLID + ' + cmbsurvey
    postpend = '_full.png'

tempnames = ['0.79', '0.85', '0.92', '0.98', '1.05', '1.11', '1.18', '1.24', '1.31', '1.37', '1.44', '1.50']
masses = np.geomspace(0.01, 10., 10)
temps = np.linspace(0.79, 1.5, 12)

datapath = '/Volumes/SSD01/results/ps19'

data = np.array([[np.sqrt(np.loadtxt(
    datapath+'/'+cmbsurvey+'/T'+tempnames[tidx]+'/M'+str(midx+1)+'/inv_'+dtype+'fisher.mat', delimiter='\t', skiprows=1)[paramidx, paramidx])
                               for midx, mval in enumerate(masses)] 
                               for tidx, tval in enumerate(temps)]) 

flatdata = data.flatten()
flatnsigmas = 1./flatdata
logflatdata = np.log10(flatdata)
logflatnsigmas = np.log10(flatnsigmas)




plt.figure(figsize=(15,7.5))

for tidx, tval in enumerate(temps): 
    plt.semilogx(masses, data[tidx, :], 'o', label=r"$\sigma_{g_\chi}$, DESI, " + str(f"{tval:.2f}"), linestyle='solid')

plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{g_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.title(title, fontsize=22)
plt.savefig(datapath+'/'+cmbsurvey+'/plotA19a'+postpend)
plt.show()

plt.figure(figsize=(25,25))    
# Create long format
people = np.repeat(np.flip(temps), len(masses))
feature = list(masses) * len(temps)
df=pd.DataFrame({'Mass [eV]': feature, 'Temperature [K]': people, 'value': flatdata })
# plot it
df_wide=df.pivot_table( index='Temperature [K]', columns='Mass [eV]', values='value' )
ax=sns.heatmap(df_wide, 
               yticklabels=(np.flip(temps)),
               #xticklabels=(np.flip(masses)),
               cbar_kws={'label': r'$\sigma_{g_\chi}$'}, 
               #vmin=0.2, 
               #vmax=1.5,
               annot=True
              )

# format text labels
fmt = '{:0.2f}'
xticklabels = []
for item in ax.get_xticklabels():
    item.set_text(fmt.format(float(item.get_text())))
    xticklabels += [item]
yticklabels = []
for item in ax.get_yticklabels():
    item.set_text(fmt.format(float(item.get_text())))
    yticklabels += [item]

ax.set_xticklabels(xticklabels, fontsize=18)
ax.set_yticklabels(yticklabels, fontsize=18)

ax.set_xlabel(ax.get_xlabel(), fontsize=18)
ax.set_ylabel(ax.get_ylabel(), fontsize=18)

cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=18)
ax.figure.axes[-1].set_ylabel(ax.figure.axes[-1].get_ylabel(), size=22)

plt.title(title, fontsize=22)
plt.savefig(datapath+'/'+cmbsurvey+'/plotA19b'+postpend)
plt.show()

# Plot Set 20 - Minimum Relic Temp Fine Plots for EUCLID

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import cosmicfish as cf
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Volumes/SSD01/data.nosync20/")
classpath = os.path.join(projectdir, "class4")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
#ps20_resultsdir = os.path.join(projectdir, 'results', 'ps20')
ps20_resultsdir = cf.correct_path("/Volumes/SSD01/results/ps20/")
#ps20_convergencedir = os.path.join(ps20_resultsdir, 'convergence')
cf.makedirectory(ps20_resultsdir)
#cf.makedirectory(ps20_convergencedir) 

# Linda Fiducial Cosmology 
ps20_fid = {
        "A_s" : 2.2321e-9,
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1.,
        "deg_ncdm" : 1,
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0,
        "beta0" : 1.7,
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# EUCLID values
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309,
    2308.975, 1514.831, 1474.707, 893.716, 497.613])
skycover = 0.3636 # Sky coverage of survey in fraction

# Run Fisher Forecast
masses = np.geomspace(0.01, 10., 21)
temps = np.array([0.79, 0.91, 0.94, 1.08])

omegacdm_set = np.array([ps20_fid['omega_cdm'] - ps20_fid["N_ncdm"]*((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(tval / 1.95, 3.)) for tidx, tval in enumerate(temps)])                 
ps20_fiducialset = [[dict(ps20_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[tidx, midx], 'T_ncdm' : temps[tidx]/2.726})
               for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
ps20_forecastset = [[cf.forecast(
    classpath,
    datastore,
    'relic',
    fidval,
    z_table,
    "EUCLID",
    dNdz,
    fsky=skycover,
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(fidrowvals)] for fidrowidx, fidrowvals in enumerate(ps20_fiducialset)]


#dill.load_session('/n/home02/ndeporzio/projects/cosmicfish/cfworkspace/results/51392653/1_12_ps20.db')
for frowidx, frowval in enumerate(ps20_forecastset):
    for fidx, fcst in enumerate(frowval):
        if type(fcst.fisher)==type(None):
            fcst.gen_pm()
            fcst.gen_fisher(
                fisher_order=[
                    'omega_b',
                    'omega_cdm',
                    'n_s',
                    'A_s',
                    'tau_reio',
                    'h',                                                                    
                    'N_ncdm',
                    'sigma_fog',
                    'beta0',
                    'beta1',
                    'alpha_k2'],
                mu_step=mu_integral_step,
                skipgen=False)
            print("Relic Forecast ", fidx, " complete...")
            dill.dump_session(os.path.join(ps20_resultsdir, str(frowidx)+'_'+str(fidx)+'_ps20.db'))
        else:
            print('Fisher matrix already generated!')
            
data = np.array([[np.sqrt(
            np.linalg.inv(
                np.delete(np.delete(fval.fisher, 4, axis=0), 4, axis=1))[5,5])
                               for fidx, fval in enumerate(frowval)]
                               for frowidx, frowval in enumerate(ps20_forecastset)])

flatdata = data.flatten()
flatnsigmas = 1./flatdata
logflatdata = np.log10(flatdata)
logflatnsigmas = np.log10(flatnsigmas)




In [ ]:


plt.figure(figsize=(15,7.5))
for tidx, tval in enumerate(temps):
    plt.semilogx(masses, data[tidx, :], 'o', label=r"EUCLID, " + str(tval) +'K', linestyle='solid')
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{g_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig(ps20_resultsdir + "/plotA20a.png")


plt.figure(figsize=(15,7.5))
for tidx, tval in enumerate(temps):
    plt.semilogx(masses, 1./data[tidx, :], 'o', label=r"EUCLID, " + str(tval) + 'K', linestyle='solid')
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$1 / \sigma_{g_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig(ps20_resultsdir + "/plotA20b.png")

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/ClusterResults_2020_04_11/ps20/52270119/3_20_ps20.db')

ps20_resultsdir = cf.correct_path("/Volumes/SSD01/results/ps20/")

#cmbsurvey="CMBS4"
cmbsurvey="Planck"

prepath = cf.priors_directory()
tempnames = ['0.79', '0.91', '0.94', '1.08']
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t N_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'


for tidx, tval in enumerate(ps20_forecastset):
    for midx, mval in enumerate(tval): 
        #mval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'N_ncdm', 'sigma_fog', 'b0', 'alpha_k2'])
        mval.load_cmb_fisher(
            fisher_order=[
                'omega_b',                                    
                'omega_cdm',                                  
                'n_s',                                        
                'A_s',                                        
                'tau_reio',                                   
                'h',                                                                             
                'N_ncdm'],
            fisherpath = cf.priors_directory()+'/Relic_Fine_Priors/Fisher_T'+tempnames[tidx]+'K/Fisher'+cmbsurvey+'_bin'+str(midx+1)+'_T'+tempnames[tidx]+'K.txt'
        )
        outdir=os.path.join(ps20_resultsdir, cmbsurvey, 'T'+tempnames[tidx], 'M'+str(midx+1))
        cf.makedirectory(outdir)
        mval.export_matrices(outdir)
        np.savetxt(outdir + '_Full_Fisher.dat', mval.numpy_full_fisher, delimiter='\t', header=head)
        mval.pandas_lss_fisher.to_csv(outdir + "_LSS_Fisher.dat", sep='\t', index=False)

# Plot Set 21 - Grid Plot for BOSS

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import cosmicfish as cf
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Volumes/SSD01/data.nosync21/")
classpath = os.path.join(projectdir, "class5")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
#ps21_resultsdir = os.path.join(projectdir, 'results', 'ps21')
ps21_resultsdir = cf.correct_path("/Volumes/SSD01/results/ps21/")
#ps21_convergencedir = os.path.join(ps21_resultsdir, 'convergence')
cf.makedirectory(ps21_resultsdir)
#cf.makedirectory(ps21_convergencedir) 

# Linda Fiducial Cosmology 
ps21_fid = {
        "A_s" : 2.2321e-9,
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1.,
        "deg_ncdm" : 1,
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0,
        "beta0" : 1.7,
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# BOSS values
z_table = np.array([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75])
dNdz = np.array([8., 50., 125., 222., 332., 447., 208., 30.])
skycover = 10000. # Sky coverage of BOSS in degrees^2

# Run Fisher Forecast
masses = np.geomspace(0.01, 10., 10)
temps = np.linspace(0.79, 1.5, 12)

omegacdm_set = np.array([ps21_fid['omega_cdm'] - ps21_fid["N_ncdm"]*((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(tval / 1.95, 3.)) for tidx, tval in enumerate(temps)])                 
ps21_fiducialset = [[dict(ps21_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[tidx, midx], 'T_ncdm' : temps[tidx]/2.726})
               for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
ps21_forecastset = [[cf.forecast(
    classpath,
    datastore,
    'relic',
    fidval,
    z_table,
    "EUCLID",
    dNdz,
    fcoverage_deg=skycover,
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(fidrowvals)] for fidrowidx, fidrowvals in enumerate(ps21_fiducialset)]


                                                 

In [ ]:
import dill
#dill.load_session('/Volumes/SSD01/results/ps21/1_0_ps21.db')
for frowidx, frowval in enumerate(ps21_forecastset):
    for fidx, fcst in enumerate(frowval):
        if type(fcst.fisher)==type(None):
            fcst.gen_pm()
            fcst.gen_fisher(
                fisher_order=[
                    'omega_b',
                    'omega_cdm',
                    'n_s',
                    'A_s',
                    'tau_reio',
                    'h',                                                                    
                    'N_ncdm',
                    'sigma_fog',
                    'beta0',
                    'beta1',
                    'alpha_k2'],
                mu_step=mu_integral_step,
                skipgen=False)
            print("Relic Forecast ", fidx, " complete...")
            dill.dump_session(os.path.join(ps21_resultsdir, str(frowidx)+'_'+str(fidx)+'_ps21.db'))
        else:
            print('Fisher matrix already generated!')
            
data = np.array([[np.sqrt(
            np.linalg.inv(
                np.delete(np.delete(fval.fisher, 4, axis=0), 4, axis=1))[5,5])
                               for fidx, fval in enumerate(frowval)]
                               for frowidx, frowval in enumerate(ps21_forecastset)])

print(data)

flatdata = data.flatten()
flatnsigmas = 1./flatdata
logflatdata = np.log10(flatdata)
logflatnsigmas = np.log10(flatnsigmas)


plt.figure(figsize=(15,7.5))
for tidx, tval in enumerate(temps):
    plt.semilogx(masses, data[tidx, :], 'ro', label=r"$\sigma_{g_\chi}$, BOSS, " + str(tval), linestyle='solid')
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{g_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig(ps21_resultsdir + "/plotA21a.png")

plt.figure(figsize=(15,15))
# Create long format
people = np.repeat(np.flip(temps), len(masses))
feature = list(masses) * len(temps)
df=pd.DataFrame({'Mass [eV]': feature, 'Temperature [K]': people, 'value': flatdata })
# plot it
df_wide=df.pivot_table( index='Temperature [K]', columns='Mass [eV]', values='value' )
p2=sns.heatmap(df_wide,
               yticklabels=(np.flip(temps)),
               #xticklabels=(np.flip(masses)),
               cbar_kws={'label': r'$\sigma_{g_\chi}$'},
               #vmin=0.2, 
               #vmax=1.5,
               annot=True
              )
plt.savefig(ps21_resultsdir + "/plotA21b.png")

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/ClusterResults_2020_04_11/ps21/52270120/11_9_ps21.db')
import cosmicfish as cf

ps21_resultsdir = cf.correct_path("/Volumes/SSD01/results/ps21/")

#cmbsurvey="CMBS4"
cmbsurvey="Planck"

prepath = cf.priors_directory()
tempnames = ['0.79', '0.85', '0.92', '0.98', '1.05', '1.11', '1.18', '1.24', '1.31', '1.37', '1.44', '1.50']
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t N_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'


for tidx, tval in enumerate(ps21_forecastset):
    for midx, mval in enumerate(tval): 
        #mval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'N_ncdm', 'sigma_fog', 'b0', 'alpha_k2'])
        mval.load_cmb_fisher(
            fisher_order=[
                'omega_b',                                    
                'omega_cdm',                                  
                'n_s',                                        
                'A_s',                                        
                'tau_reio',                                   
                'h',                                                                             
                'N_ncdm'],
            fisherpath = cf.priors_directory()+'/Relic_Grid_Priors/Fisher_T'+tempnames[tidx]+'K/Fisher'+cmbsurvey+'_bin'+str(midx+1)+'_T'+tempnames[tidx]+'K.txt'
        )
        outdir=os.path.join(ps21_resultsdir, cmbsurvey, 'T'+tempnames[tidx], 'M'+str(midx+1))
        cf.makedirectory(outdir)
        mval.export_matrices(outdir)
        np.savetxt(outdir + '_Full_Fisher.dat', mval.numpy_full_fisher, delimiter='\t', header=head)
        mval.pandas_lss_fisher.to_csv(outdir + "_LSS_Fisher.dat", sep='\t', index=False)

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt

#cmbsurvey="CMBS4"
cmbsurvey="Planck"

#dtype="cmb"
#dtype="lss"
dtype="full"

paramidx = 6

if dtype=='lss': 
    title = 'BOSS Only'
    postpend = '_lss.png'
elif dtype=='cmb': 
    title =  cmbsurvey + ' Only'
    postpend = '_cmb.png'
elif dtype=='full': 
    title = 'BOSS + ' + cmbsurvey
    postpend = '_full.png'

tempnames = ['0.79', '0.85', '0.92', '0.98', '1.05', '1.11', '1.18', '1.24', '1.31', '1.37', '1.44', '1.50']
masses = np.geomspace(0.01, 10., 10)
temps = np.linspace(0.79, 1.5, 12)

datapath = '/Volumes/SSD01/results/ps21'

data = np.array([[np.sqrt(np.loadtxt(
    datapath+'/'+cmbsurvey+'/T'+tempnames[tidx]+'/M'+str(midx+1)+'/inv_'+dtype+'fisher.mat', delimiter='\t', skiprows=1)[paramidx, paramidx])
                               for midx, mval in enumerate(masses)] 
                               for tidx, tval in enumerate(temps)]) 

flatdata = data.flatten()
flatnsigmas = 1./flatdata
logflatdata = np.log10(flatdata)
logflatnsigmas = np.log10(flatnsigmas)




plt.figure(figsize=(15,7.5))

for tidx, tval in enumerate(temps): 
    plt.semilogx(masses, data[tidx, :], 'o', label=r"$\sigma_{g_\chi}$, DESI, " + str(f"{tval:.2f}"), linestyle='solid')

plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{g_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.title(title, fontsize=22)
plt.savefig(datapath+'/'+cmbsurvey+'/plotA21a'+postpend)
plt.show()

plt.figure(figsize=(25,25))    
# Create long format
people = np.repeat(np.flip(temps), len(masses))
feature = list(masses) * len(temps)
df=pd.DataFrame({'Mass [eV]': feature, 'Temperature [K]': people, 'value': flatdata })
# plot it
df_wide=df.pivot_table( index='Temperature [K]', columns='Mass [eV]', values='value' )
ax=sns.heatmap(df_wide, 
               yticklabels=(np.flip(temps)),
               #xticklabels=(np.flip(masses)),
               cbar_kws={'label': r'$\sigma_{g_\chi}$'}, 
               #vmin=0.2, 
               #vmax=1.5,
               annot=True
              )

# format text labels
fmt = '{:0.2f}'
xticklabels = []
for item in ax.get_xticklabels():
    item.set_text(fmt.format(float(item.get_text())))
    xticklabels += [item]
yticklabels = []
for item in ax.get_yticklabels():
    item.set_text(fmt.format(float(item.get_text())))
    yticklabels += [item]

ax.set_xticklabels(xticklabels, fontsize=18)
ax.set_yticklabels(yticklabels, fontsize=18)

ax.set_xlabel(ax.get_xlabel(), fontsize=18)
ax.set_ylabel(ax.get_ylabel(), fontsize=18)

cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=18)
ax.figure.axes[-1].set_ylabel(ax.figure.axes[-1].get_ylabel(), size=22)

plt.title(title, fontsize=22)
plt.savefig(datapath+'/'+cmbsurvey+'/plotA21b'+postpend)
plt.show()

# Plot Set 22 - Minimum Relic Temp Fine Plots for BOSS

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import cosmicfish as cf
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Volumes/SSD01/data.nosync22/")
classpath = os.path.join(projectdir, "class6")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
#ps22_resultsdir = os.path.join(projectdir, 'results', 'ps22')
ps22_resultsdir = cf.correct_path("/Volumes/SSD01/results/ps22/")
#ps22_convergencedir = os.path.join(ps22_resultsdir, 'convergence')
cf.makedirectory(ps22_resultsdir)
#cf.makedirectory(ps22_convergencedir) 

# Linda Fiducial Cosmology 
ps22_fid = {
        "A_s" : 2.2321e-9,
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1.,
        "deg_ncdm" : 1,
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0,
        "beta0" : 1.7,
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# BOSS values
z_table = np.array([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75])
dNdz = np.array([8., 50., 125., 222., 332., 447., 208., 30.])
skycover = 10000. # Sky coverage of BOSS in degrees^2

# Run Fisher Forecast
masses = np.geomspace(0.01, 10., 21)
temps = np.array([0.79, 1.5])

omegacdm_set = np.array([ps22_fid['omega_cdm'] - ps22_fid["N_ncdm"]*((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(tval / 1.95, 3.)) for tidx, tval in enumerate(temps)])                 
ps22_fiducialset = [[dict(ps22_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[tidx, midx], 'T_ncdm' : temps[tidx]/2.726})
               for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
ps22_forecastset = [[cf.forecast(
    classpath,
    datastore,
    'relic',
    fidval,
    z_table,
    "EUCLID",
    dNdz,
    fcoverage_deg=skycover,
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(fidrowvals)] for fidrowidx, fidrowvals in enumerate(ps22_fiducialset)]

#dill.load_session('/n/home02/ndeporzio/projects/cosmicfish/cfworkspace/results/51392655/3_5_ps22.db')
for frowidx, frowval in enumerate(ps22_forecastset):
    for fidx, fcst in enumerate(frowval):
        if type(fcst.fisher)==type(None):
            fcst.gen_pm()
            fcst.gen_fisher(
                fisher_order=[
                    'omega_b',
                    'omega_cdm',
                    'n_s',
                    'A_s',
                    'tau_reio',
                    'h',                                                                    
                    'N_ncdm',
                    'sigma_fog',
                    'beta0',
                    'beta1',
                    'alpha_k2'],
                mu_step=mu_integral_step,
                skipgen=False)
            print("Relic Forecast ", fidx, " complete...")
            dill.dump_session(os.path.join(ps22_resultsdir, str(frowidx)+'_'+str(fidx)+'_ps22.db'))
        else:
            print('Fisher matrix already generated!')
            


In [ ]:
import dill
dill.load_session('/Volumes/SSD01/results/ps22/loginterp/1_0_ps22.db')

data = np.array([[np.sqrt(
            np.linalg.inv(
                np.delete(np.delete(fval.fisher, 4, axis=0), 4, axis=1))[5,5])
                               for fidx, fval in enumerate(frowval)]
                               for frowidx, frowval in enumerate(ps22_forecastset)])

print(data)

flatdata = data.flatten()
flatnsigmas = 1./flatdata
logflatdata = np.log10(flatdata)
logflatnsigmas = np.log10(flatnsigmas)


plt.figure(figsize=(15,7.5))
for tidx, tval in enumerate(temps):
    plt.semilogx(masses, data[tidx, :], 'o', label=r"BOSS, " + str(tval) +'K', linestyle='solid')
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$\sigma_{g_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig(ps22_resultsdir + "/plotA22a.png")


plt.figure(figsize=(15,7.5))
for tidx, tval in enumerate(temps):
    plt.semilogx(masses, 1./data[tidx, :], 'o', label=r"BOSS, " + str(tval) + 'K', linestyle='solid')
plt.xlabel(r"$M_\chi$ [eV]", fontsize=24)
plt.ylabel(r"$1 / \sigma_{g_\chi}$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig(ps22_resultsdir + "/plotA22b.png")

In [ ]:
import dill
dill.load_session('/Users/nicholasdeporzio/Desktop/ClusterResults_2020_04_11/ps22/52270121/3_20_ps22.db')
import cosmicfish as cf

ps22_resultsdir = cf.correct_path("/Volumes/SSD01/results/ps22/")

#cmbsurvey="CMBS4"
cmbsurvey="Planck"

prepath = cf.priors_directory()
tempnames = ['0.79', '0.91', '0.94', '1.08']
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t N_ncdm \t sigma_fog \t beta0 \t beta1 \t alpha_k2'


for tidx, tval in enumerate(ps22_forecastset):
    for midx, mval in enumerate(tval): 
        #mval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'N_ncdm', 'sigma_fog', 'b0', 'alpha_k2'])
        mval.load_cmb_fisher(
            fisher_order=[
                'omega_b',                                    
                'omega_cdm',                                  
                'n_s',                                        
                'A_s',                                        
                'tau_reio',                                   
                'h',                                                                             
                'N_ncdm'],
            fisherpath = cf.priors_directory()+'/Relic_Fine_Priors/Fisher_T'+tempnames[tidx]+'K/Fisher'+cmbsurvey+'_bin'+str(midx+1)+'_T'+tempnames[tidx]+'K.txt'
        )
        outdir=os.path.join(ps22_resultsdir, cmbsurvey, 'T'+tempnames[tidx], 'M'+str(midx+1))
        cf.makedirectory(outdir)
        mval.export_matrices(outdir)
        np.savetxt(outdir + '_Full_Fisher.dat', mval.numpy_full_fisher, delimiter='\t', header=head)
        mval.pandas_lss_fisher.to_csv(outdir + "_LSS_Fisher.dat", sep='\t', index=False)

# Save Cl Datasets

In [ ]:
import os
import shutil

varnames = ['A_s', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'h', 'N_ncdm']
for frowidx, frowval in enumerate(ps18_forecastset): 
    for fidx, fval  in enumerate(frowval): 
        mass = ps18_fiducialset[frowidx][fidx]['m_ncdm']
        temp = ps18_fiducialset[frowidx][fidx]['T_ncdm']*2.726
        omega_b_high = fval.omega_b_high[0].datadirectory
        omega_b_low = fval.omega_b_low[0].datadirectory
        omega_cdm_high = fval.omega_cdm_high[0].datadirectory
        omega_cdm_low = fval.omega_cdm_low[0].datadirectory
        h_high = fval.h_high[0].datadirectory
        h_low = fval.h_low[0].datadirectory
        tau_reio_high = fval.tau_reio_high[0].datadirectory
        tau_reio_low = fval.tau_reio_low[0].datadirectory
        # T_ncdm_high = fval.T_ncdm_high[0].datadirectory
        # T_ncdm_low = fval.T_ncdm_low[0].datadirectory
        N_ncdm_high = fval.N_ncdm_high[0].datadirectory
        N_ncdm_low = fval.N_ncdm_low[0].datadirectory
        A_s_high = fval.A_s_high[0].datadirectory
        A_s_low = fval.A_s_low[0].datadirectory
        n_s_high = fval.n_s_high[0].datadirectory
        n_s_low = fval.n_s_low[0].datadirectory
        for pidx, pval  in enumerate(varnames): 
            path = ('/Volumes/SSD01/Cls/Fine_Cls/T_ncdm_'+str(temp)+'/m_ncdm_' + str(mass) + '/' + pval + '/')
            print('From: ' + eval((pval+'_high')))
            print('To: ' +path+'high')
            shutil.copytree(eval((pval+'_high')), (path+'high'))
            shutil.copytree(eval((pval+'_low')), (path+'low'))
